<a href="https://colab.research.google.com/github/Tanmay3463/SWEET_algo_trading_software/blob/main/Angelone_v4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
!git clone https://github.com/angel-one/smartapi-python.git
!pip install pyotp logzero websocket-client pycryptodome python-dotenv requests

fatal: destination path 'smartapi-python' already exists and is not an empty directory.


# **Imports**

In [33]:
import sys
sys.path.append('/content/smartapi-python')

import os
import pyotp
import pandas as pd
import requests
from dotenv import load_dotenv
from logzero import logger
from SmartApi import SmartConnect
from datetime import datetime, timedelta
import gradio as gr
import time
import json
import matplotlib.pyplot as plt
import numpy as np
import logging
import math
import shutil

# --- Global variables for separate exchange maps ---
nse_symbol_token_map = {}
bse_symbol_token_map = {}
nse_symbols_list = []
bse_symbols_list = []

_industry_df_cache = None

In [34]:
directory_path = "/content/downloadrohit/backup/"
os.makedirs(directory_path, exist_ok=True)

directory_path2 = "/content/masterrohit/"
os.makedirs(directory_path2, exist_ok=True)

# **Login**

In [35]:

# --- Setup Logging ---
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# --- Load .env and Initialize API (User's Code) ---
load_dotenv()
api_key = os.getenv("API_KEY_TRADING")
username = os.getenv("USERNAME")
pwd = os.getenv("PASSWORD")
totp_token = os.getenv("TOTP_TOKEN")

smartApi = None
login_status_message = ""

if not all([api_key, username, pwd, totp_token]):
    login_status_message = "❗️ Missing API_KEY, USERNAME, PASSWORD, or TOTP_TOKEN in .env file"
    logging.error(login_status_message)
else:
    smartApi = SmartConnect(api_key)
    try:
        totp = pyotp.TOTP(totp_token).now()
        session_data = smartApi.generateSession(username, pwd, totp)
        if session_data and session_data.get('status'):
             login_status_message = "✅ Login successful!"
             logging.info(login_status_message)
        else:
            login_status_message = f"❌ Login failed: {session_data.get('message', 'Unknown error')}"
            logging.error(login_status_message)
            smartApi = None
    except Exception as e:
        login_status_message = f"❌ Login failed with an exception: {e}"
        logging.error(login_status_message)
        smartApi = None

[I 250927 07:11:06 smartConnect:124] in pool


In [36]:
api_key_history = os.getenv("API_KEY_HISTORY")
username = os.getenv("USERNAME")
pwd = os.getenv("PASSWORD")
totp_token = os.getenv("TOTP_TOKEN")

if not all([api_key_history, username, pwd, totp_token]):
    raise ValueError("Missing API_KEY, USERNAME, PASSWORD, or TOTP_TOKEN in .env file")

smartApi_hist = SmartConnect(api_key_history)

try:
    totp_hist = pyotp.TOTP(totp_token).now()
except Exception as e:
    logger.error("Invalid TOTP Token")
    raise e

try:
    session_data_hist = smartApi_hist.generateSession(username, pwd, totp)
    logger.info("✅ Login successful!")
except Exception as e:
    logger.error(f"❌ Login failed: {e}")

print(totp)

[I 250927 07:11:07 smartConnect:124] in pool
[I 250927 07:11:09 ipython-input-1567320128:19] ✅ Login successful!


739544


In [37]:
nse = "downloadrohit/script_master_NSE.csv"
bse = "downloadrohit/script_master_BSE.csv"
portfolio = "downloadrohit/portfolio_holdings.csv"
order_book = "downloadrohit/order_book.csv"
trade_book = "downloadrohit/trade_book.csv"
rule_book = 'masterrohit/Rule_book.csv'
industry = "masterrohit/Dashboard_Industry.csv"
master_portfolio = 'masterrohit/master_portfolio.csv'
archive = 'masterrohit/archive.csv'
SWEET_Code_path = 'masterrohit/SWEET_Code.csv'
SLEEP = 0.1

In [38]:
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Constants for the URL and local file paths
JSON_URL = "https://margincalculator.angelbroking.com/OpenAPI_File/files/OpenAPIScripMaster.json"
NSE_CSV_PATH = nse
BSE_CSV_PATH = bse


def _fetch_from_url(url: str) -> list | None:
    """
    Attempts to download and parse the instrument master JSON from the given URL.
    Returns the JSON data as a list of dictionaries, or None if it fails.
    """
    logging.info(f"Attempting to download script master from {url}")
    try:
        # Added a timeout for robustness
        response = requests.get(url, timeout=10)
        response.raise_for_status()  # Raises an HTTPError for bad responses (4xx or 5xx)
        return response.json()
    except requests.exceptions.RequestException as e:
        logging.error(f"Error during download: {e}")
        return None
    except ValueError as e:
        logging.error(f"Error decoding JSON: {e}")
        return None

def _process_and_save_data(script_data: list) -> tuple[list, list, dict]:
    """
    Processes the raw script data, saves it to separate CSVs for NSE and BSE,
    and returns the symbols lists and token map.
    """
    if not script_data:
        logging.warning("Received empty script data. Cannot process.")
        return [], [], {}

    df = pd.DataFrame(script_data)

    # Filter data for NSE and BSE
    nse_df = df[df['exch_seg'] == 'NSE'].copy()
    bse_df = df[df['exch_seg'] == 'BSE'].copy()

    # Save to CSV files
    nse_df.to_csv(NSE_CSV_PATH, index=False)
    logging.info(f"Successfully saved {len(nse_df)} NSE instruments to {NSE_CSV_PATH}")
    bse_df.to_csv(BSE_CSV_PATH, index=False)
    logging.info(f"Successfully saved {len(bse_df)} BSE instruments to {BSE_CSV_PATH}")

    # Create the required data structures
    nse_symbols = sorted(nse_df['symbol'].dropna().unique().tolist())
    bse_symbols = sorted(bse_df['symbol'].dropna().unique().tolist())

    # Create mapping with exchange key to avoid clashes
    # Drop rows where key components are missing to prevent errors
    token_df = df.dropna(subset=['exch_seg', 'symbol', 'token'])
    token_map = {
        (row.exch_seg, row.symbol): row.token
        for row in token_df.itertuples()
    }

    return nse_symbols, bse_symbols, token_map

def _load_from_csv() -> tuple[list, list, dict]:
    """
    Loads instrument data from local CSV files as a fallback.
    Returns the symbols lists and token map.
    """
    if not os.path.exists(NSE_CSV_PATH) or not os.path.exists(BSE_CSV_PATH):
        logging.error("Fallback CSV files not found. Cannot load data.")
        return [], [], {}

    try:
        logging.info(f"Loading data from local files: {NSE_CSV_PATH} and {BSE_CSV_PATH}")
        nse_df = pd.read_csv(NSE_CSV_PATH)
        bse_df = pd.read_csv(BSE_CSV_PATH)

        # Combine for the token map
        combined_df = pd.concat([nse_df, bse_df])

        nse_symbols = sorted(nse_df['symbol'].dropna().unique().tolist())
        bse_symbols = sorted(bse_df['symbol'].dropna().unique().tolist())

        token_df = combined_df.dropna(subset=['exch_seg', 'symbol', 'token'])
        token_map = {
            (row.exch_seg, row.symbol): row.token
            for row in token_df.itertuples()
        }

        return nse_symbols, bse_symbols, token_map
    except Exception as e:
        logging.error(f"Failed to load or process data from CSV files: {e}")
        return [], [], {}

def load_instrument_master() -> tuple[list, list, dict]:

    script_data = _fetch_from_url(JSON_URL)

    if script_data:
        # If download was successful, process and save the new data
        return _process_and_save_data(script_data)
    else:
        # If download failed, trigger the fallback
        logging.warning("Download failed. Attempting to load from local cache.")
        return _load_from_csv()

if __name__ == "__main__":
    # This is how you would use the function in your main script
    nse_symbols, bse_symbols, token_map = load_instrument_master()

    if token_map:
        logging.info(f"Loaded {len(nse_symbols)} NSE and {len(bse_symbols)} BSE symbols.")

        # Example: Look up the token for a specific stock
        reliance_token = token_map.get(('NSE', 'RELIANCE-EQ'))
        if reliance_token:
            logging.info(f"The token for RELIANCE-EQ on NSE is: {reliance_token}")
        else:
            logging.warning("Could not find token for RELIANCE-EQ on NSE.")

        # Example: Check the first 5 symbols in the NSE list
        logging.info(f"First 5 NSE symbols: {nse_symbols[:5]}")
    else:
        logging.error("Failed to load instrument master from all sources.")


In [39]:
# Default date range
today = datetime.now()
fifty_two_weeks_ago = today - timedelta(weeks=52)
default_from_date = fifty_two_weeks_ago.strftime("%Y-%m-%d %H:%M")
default_to_date = today.strftime("%Y-%m-%d %H:%M")

# **Historical Data**

In [40]:
def update_symbol_dropdown(exchange):
    return gr.Dropdown(choices=nse_symbols if exchange == "NSE" else bse_symbols, multiselect=True, interactive=True)

def select_all_symbols(exchange):
    return nse_symbols if exchange == "NSE" else bse_symbols

In [41]:
def get_historical_data_for_tokens(symbol_tokens, exchange, interval, fromdate, todate):
    all_candle_data = {}
    # Create a mapping from token back to symbol for easier lookup
    token_to_symbol = {token: symbol for (exch, symbol), token in token_map.items() if exch == exchange and token in symbol_tokens}

    for token in symbol_tokens:
        params = {
            "exchange": exchange,
            "symboltoken": token,
            "interval": interval,
            "fromdate": fromdate,
            "todate": todate
        }
        logger.info(f"📤 Requesting data with params: {params}")

        try:
            candle_data = smartApi_hist.getCandleData(params)
            logger.info(f"📥 Response: {candle_data}")

            if candle_data and candle_data.get('status') and candle_data.get('data'):
                df = pd.DataFrame(candle_data['data'], columns=['DateTime', 'Open', 'High', 'Low', 'Close', 'Volume'])
                symbol = token_to_symbol.get(token, "Unknown Symbol")
                df['Token Symbol'] = symbol
                all_candle_data[token] = df
            else:
                logger.warning(f"⚠️ No data for token: {token}")
        except Exception as e:
            logger.exception(f"❌ Failed for token {token}: {e}")
        time.sleep(SLEEP*2)
    return all_candle_data

In [42]:
def get_data_and_prepare_download(input_type, selected_symbols, uploaded_file, exchange, interval, fromdate, todate):
    if input_type == "Manual Selection":
        if not selected_symbols:
            return pd.DataFrame(), None, "Please select at least one symbol."
        symbols_to_process = selected_symbols
    elif input_type == "Upload Excel File":
        if not uploaded_file:
            return pd.DataFrame(), None, "Please upload an Excel file."
        try:
            df_upload = pd.read_excel(uploaded_file.name)
            if 'Token Symbol' not in df_upload.columns:
                return pd.DataFrame(), None, "Uploaded file must contain a 'Token Symbol' column."
            symbols_to_process = df_upload['Token Symbol'].tolist()
        except Exception as e:
            return pd.DataFrame(), None, f"Error reading Excel file: {e}"
    else:
        return pd.DataFrame(), None, "Invalid input method selected."

    # Process symbols to get unique list and remove any potential non-string values
    symbols_to_process = [s for s in symbols_to_process if isinstance(s, str)]
    symbols_to_process = list(set(symbols_to_process))

    if not symbols_to_process:
        return pd.DataFrame(), None, "No valid symbols found in the selected or uploaded input."

    try:
        tokens = [token_map.get((exchange, s)) for s in symbols_to_process if (exchange, s) in token_map]
        if not tokens:
            return pd.DataFrame(), None, f"Could not find tokens for selected symbols in {exchange}."

        all_data = get_historical_data_for_tokens(tokens, exchange, interval, fromdate, todate)

        if not all_data:
            return pd.DataFrame(), None, "No historical data retrieved for the selected symbols and date range."

        combined_df = pd.concat(all_data.values(), ignore_index=True)

        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        csv_filename = f"downloadrohit/historical_data_{timestamp}.csv"
        combined_df.to_csv(csv_filename, index=False)

        return combined_df, csv_filename, "Data retrieved and ready for download."

    except Exception as e:
        logger.exception(f"An error occurred during data retrieval or processing: {e}")
        return pd.DataFrame(), None, f"An error occurred: {e}"


# **Master File**

In [43]:
def download_script_master_csv(exchange: str) -> tuple[str | None, str]:
    """
    Downloads and filters the script master data based on the selected exchange,
    leveraging the efficient caching mechanism.

    This function maintains the original signature for Gradio compatibility but
    is powered by the robust load_instrument_master() function to avoid
    redundant downloads. It saves the final output to a 'downloadrohit/' subdirectory.

    Args:
        exchange (str): The exchange to get the CSV for. Typically 'NSE' or 'BSE'.

    Returns:
        A tuple containing:
        - file_path (str | None): The path to the CSV file, or None on failure.
        - message (str): A status message.
    """
    # First, ensure the data is loaded and caches are up-to-date.
    nse_symbols, bse_symbols, token_map = load_instrument_master()

    if not token_map:
        return None, "Failed to load instrument master data from all sources."

    # Determine the correct source file path based on the exchange
    if exchange == "NSE":
        source_path = NSE_CSV_PATH
    elif exchange == "BSE":
        source_path = BSE_CSV_PATH
    else:
        message = f"Invalid exchange '{exchange}'. Please use 'NSE' or 'BSE'."
        logging.error(message)
        return None, message

    # Define the target directory and filename
    target_dir = "downloadrohit"
    target_path = os.path.join(target_dir, f"script_master_{exchange}.csv")

    try:
        # Ensure the target directory exists
        os.makedirs(target_dir, exist_ok=True)

        # Read the source cache file and save it to the target location
        df = pd.read_csv(source_path)
        df.to_csv(target_path, index=False)

        message = f"Script master for {exchange} successfully created at {target_path}"
        logging.info(message)
        return target_path, message

    except FileNotFoundError:
        message = f"Cache file {source_path} not found. Cannot create derivative CSV."
        logging.error(message)
        return None, message
    except Exception as e:
        message = f"An error occurred while creating {target_path}: {e}"
        logging.exception(message)
        return None, message

In [44]:
def toggle_input(choice):
    if choice == "Manual Selection":
        return gr.update(visible=True), gr.update(visible=True), gr.update(visible=False), gr.update(visible=True)
    else:
        return gr.update(visible=False), gr.update(visible=False), gr.update(visible=True), gr.update(visible=False)

# **Portfolio**

In [45]:
def get_portfolio_holdings():
    try:
        portfolio_data = smartApi.holding()
        if portfolio_data and portfolio_data.get('status'):
            if 'data' in portfolio_data and portfolio_data['data']:
                df = pd.DataFrame(portfolio_data['data'])
                df = df.sort_values(by='tradingsymbol')

                # Save CSV
                csv_path = portfolio
                df.to_csv(csv_path, index=False)

                # Return records for DataFrame UI + CSV path
                return df, csv_path
            else:
                df = pd.DataFrame([{"Message": "No holdings found"}])
                return df, None
        else:
            df = pd.DataFrame([{"Message": portfolio_data.get('message', 'Unknown error')}])
            return df, None
    except Exception as e:
        logger.error(f"An error occurred while fetching portfolio data: {e}")
        return pd.DataFrame([{"Error": str(e)}]), None


# **Order History**

In [46]:
def load_script_masters():
    """Loads the NSE and BSE script master files from local CSVs."""
    global nse_symbol_token_map, bse_symbol_token_map

    nse_symbol_token_map, bse_symbol_token_map = {}, {}  # Always initialize

    nse_file = nse
    bse_file = bse

    # Load NSE master
    if os.path.exists(nse_file):
        logging.info("Loading NSE script master from local file...")
        nse_master_df = pd.read_csv(nse_file)
        if not nse_master_df.empty:
            nse_master_df['token'] = nse_master_df['token'].astype(str)
            nse_symbol_token_map = pd.Series(
                nse_master_df.token.values,
                index=nse_master_df.symbol
            ).to_dict()
            logging.info("NSE Symbol-to-Token map created.")
    else:
        logging.warning(f"'{nse_file}' not found. NSE lookups will fail.")

    # Load BSE master
    if os.path.exists(bse_file):
        logging.info("Loading BSE script master from local file...")
        bse_master_df = pd.read_csv(bse_file)
        if not bse_master_df.empty:
            bse_master_df['token'] = bse_master_df['token'].astype(str)
            bse_symbol_token_map = pd.Series(
                bse_master_df.token.values,
                index=bse_master_df.symbol
            ).to_dict()
            logging.info("BSE Symbol-to-Token map created.")
    else:
        logging.warning(f"'{bse_file}' not found. BSE lookups will fail.")

def get_order_book():
    if not smartApi: return pd.DataFrame([{"Error": "Not connected to API."}])
    try:
        order_data = smartApi.orderBook()
        if order_data and order_data.get('status') and order_data.get('data') is not None:
            return pd.DataFrame(order_data['data'])
        else:
            return pd.DataFrame([{"Message": order_data.get('message', 'Order book is empty or error occurred')}])
    except Exception as e:
        logging.error(f"Error fetching order book: {e}")
        return pd.DataFrame([{"Error": str(e)}])

def get_trade_book():
    if not smartApi: return pd.DataFrame([{"Error": "Not connected to API."}])
    try:
        trade_data = smartApi.tradeBook()
        if trade_data and trade_data.get('status') and trade_data.get('data') is not None:
            return pd.DataFrame(trade_data['data'])
        else:
            return pd.DataFrame([{"Message": trade_data.get('message', 'Trade book is empty or error occurred')}])
    except Exception as e:
        logging.error(f"Error fetching trade book: {e}")
        return pd.DataFrame([{"Error": str(e)}])

# **Dashboard**

In [47]:
# --- Helper Functions ---

def _load_industry_data() -> pd.DataFrame | None:
    """
    Loads the industry data from the CSV file.
    This function is modified to always read from the file
    to reflect changes made in the Watchlist Manager.
    """
    try:
        logging.info("Loading industry data from Dashboard_Industry.csv.")
        df = pd.read_csv(
            industry,
            dtype={"BSE Code": str, "NSE Code": str}
        )

        # Clean the relevant columns once upon loading
        for col in ["BSE Code", "NSE Code", "Industry"]:
            if col in df.columns:
                df[col] = df[col].astype(str).str.strip().str.upper()

        # We are no longer caching this data persistently in a global variable
        return df

    except FileNotFoundError:
        logging.error("Dashboard_Industry.csv not found. Industry data will not be available.")
        return None
    except Exception as e:
        logging.error(f"Failed to load or process Dashboard_Industry.csv: {e}")
        return None

def _add_value_columns(df: pd.DataFrame) -> pd.DataFrame:
    """
    Adds 'invested_value' and 'current_value' columns if they don't exist.
    This prevents redundant calculations in downstream functions.
    """
    if "invested_value" not in df.columns:
        df["invested_value"] = pd.to_numeric(df["quantity"], errors='coerce') * pd.to_numeric(df["averageprice"], errors='coerce')
    if "current_value" not in df.columns:
        df["current_value"] = pd.to_numeric(df["quantity"], errors='coerce') * pd.to_numeric(df["ltp"], errors='coerce')
    return df

def _create_placeholder_fig(message="No data to plot"):
    """Creates a matplotlib figure with a text message."""
    fig, ax = plt.subplots()
    ax.text(0.5, 0.5, message, ha='center', va='center', fontsize=12)
    ax.axis('off')
    plt.tight_layout()
    return fig

# ====== Main Portfolio Function (Refined) ======

def get_portfolio_data():
    """
    Fetches portfolio holdings, cleans the data, and merges it with industry information.
    """
    try:
        portfolio_data = smartApi.holding()

        # Simplified check for valid, non-empty data
        if not (portfolio_data and portfolio_data.get('status') and portfolio_data.get('data')):
            msg = portfolio_data.get('message', 'No holdings found or an API error occurred.')
            logging.warning(msg)
            return pd.DataFrame([{"Message": msg}])

        df = pd.DataFrame(portfolio_data['data'])

        # Ensure required columns exist
        required_cols = [
            "tradingsymbol", "exchange", "symboltoken", "quantity",
            "averageprice", "ltp", "profitandloss", "pnlpercentage"
        ]
        if not all(col in df.columns for col in required_cols):
            logging.error("API response is missing one or more required columns.")
            return pd.DataFrame([{"Error": "Malformed API response from holdings endpoint."}])

        df = df[required_cols]

        # --- Refined Industry Mapping ---
        # Load industry data fresh each time to ensure latest watchlist changes are reflected
        industry_df = _load_industry_data()
        if industry_df is not None:
            # Prepare portfolio df for merging
            df['bse_code_str'] = df['symboltoken'].astype(str).str.strip().str.upper()
            df['nse_code_str'] = df['tradingsymbol'].str.replace(r'-[A-Z0-9]+$', '', regex=True).str.strip().str.upper()

            # Merge based on exchange
            df_nse = df[df['exchange'] == 'NSE']
            df_bse = df[df['exchange'] == 'BSE']

            merged_nse = pd.merge(df_nse, industry_df, left_on='nse_code_str', right_on='NSE Code', how='left')
            merged_bse = pd.merge(df_bse, industry_df, left_on='bse_code_str', right_on='BSE Code', how='left')

            df = pd.concat([merged_nse, merged_bse], ignore_index=True)
            df['Industry'] = df['Industry'].fillna('N/A') # Fill missing industries
        else:
            df['Industry'] = 'N/A' # Add placeholder if industry file is missing

        # --- Clean up and Reorder Columns ---
        df.drop(
            columns=['symboltoken', 'bse_code_str', 'nse_code_str', 'BSE Code', 'NSE Code', 'Name'],
            inplace=True,
            errors='ignore'
        )

        # Move 'Industry' column to be next to 'tradingsymbol'
        if "Industry" in df.columns:
            industry_col = df.pop("Industry")
            df.insert(loc=df.columns.get_loc("tradingsymbol") + 1, column="Industry", value=industry_col)

        # Add invested and current value columns for the final table
        df = _add_value_columns(df)

        return df

    except Exception as e:
        logging.exception("An unhandled error occurred while fetching portfolio data.")
        return pd.DataFrame([{"Error": str(e)}])

# ====== Dashboard Helper Functions (Refined) ======

def calculate_metrics(df: pd.DataFrame):

    if "Error" in df.columns or "Message" in df.columns:
        return df # Pass through error/message dataframes

    df = _add_value_columns(df.copy()) # Use .copy() to avoid SettingWithCopyWarning

    total_symbols = df["tradingsymbol"].nunique()
    total_invested = df["invested_value"].sum()
    total_current = df["current_value"].sum()
    profit_loss = total_current - total_invested
    pct_gain_loss = (profit_loss / total_invested) * 100 if total_invested else 0

    summary = {
        "Total Unique Symbols": int(total_symbols),
        "Total Invested (₹)": f"{int(round(total_invested)):,}",
        "Portfolio Value (₹)": f"{int(round(total_current)):,}",
        "Profit/Loss (₹)": f"{int(round(profit_loss)):,}",
        "P/L %": f"{pct_gain_loss:.2f}%"
    }
    return pd.DataFrame([summary])


def plot_charts(df: pd.DataFrame):

    if "Error" in df.columns or "Message" in df.columns or "Industry" not in df.columns or df['Industry'].nunique() <= 1:
        logging.warning("Cannot generate industry plots due to error, message, or insufficient data.")
        return _create_placeholder_fig(), _create_placeholder_fig()

    df = _add_value_columns(df.copy()) # Use .copy()

    industry_invested = df.groupby("Industry")["invested_value"].sum()
    industry_current = df.groupby("Industry")["current_value"].sum()

    # --- Pie Chart: Invested Value by Industry ---
    fig1, ax1 = plt.subplots(figsize=(7, 6), constrained_layout=True)

    def autopct_format(values):
        def inner_autopct(pct):
            total = sum(values)
            val = int(round(pct * total / 100.0))
            return f"{pct:.1f}%\n(₹{val:,})"
        return inner_autopct

    cmap = plt.get_cmap("tab20")
    colors = cmap.colors[:len(industry_invested)]

    wedges, texts, autotexts = ax1.pie(
        industry_invested,
        colors=colors,
        autopct=autopct_format(industry_invested),
        startangle=90,
        pctdistance=0.8,
        textprops={'fontsize': 9}
    )
    ax1.set_title("Invested Value by Industry", fontsize=14, weight="bold")
    ax1.legend(
        wedges, industry_invested.index, title="Industry",
        loc="center left", bbox_to_anchor=(1.05, 0.5), fontsize=9
    )

    # --- Bar Chart: PnL % by Industry (Replaces the old chart) ---
    fig2, ax2 = plt.subplots(figsize=(8, 6), constrained_layout=True)

    # Calculate PnL % for each industry
    industry_pnl = ((industry_current - industry_invested) / industry_invested) * 100
    industry_pnl = industry_pnl.replace([np.inf, -np.inf], 0).fillna(0) # Handle division by zero
    industry_pnl = industry_pnl.sort_values(ascending=False)

    colors = industry_pnl.apply(lambda x: 'green' if x >= 0 else 'red')

    ax2.bar(industry_pnl.index, industry_pnl, color=colors)
    ax2.set_title("Profit/Loss % by Industry", fontsize=14, weight="bold")
    ax2.set_xticklabels(industry_pnl.index, rotation=45, ha="right", fontsize=9)
    ax2.set_ylabel("P/L %")
    ax2.set_xlabel(None)
    ax2.axhline(0, color='black', linewidth=0.8) # Zero line
    ax2.grid(axis='y', linestyle='--', alpha=0.7)

    if not industry_pnl.empty:
        abs_max = max(abs(industry_pnl.min()), abs(industry_pnl.max()))
        # Set the y-axis limits with a 10% buffer for spacing
        ax2.set_ylim(-abs_max * 1.1, abs_max * 1.1)

    return fig1, fig2

def plot_pnl_bars(df: pd.DataFrame):

    if "Error" in df.columns or "Message" in df.columns or "pnlpercentage" not in df.columns:
        logging.warning("Cannot generate PnL bar chart due to error or missing data.")
        return _create_placeholder_fig()

    # Ensure data is numeric and sorted for better visualization
    df = df.copy()
    df['pnlpercentage'] = pd.to_numeric(df['pnlpercentage'], errors='coerce').fillna(0)
    df = df.sort_values('pnlpercentage', ascending=False)

    fig, ax = plt.subplots(figsize=(max(8, len(df) * 0.6), 5), constrained_layout=True)
    colors = df['pnlpercentage'].apply(lambda x: 'green' if x >= 0 else 'red')

    ax.bar(df['tradingsymbol'], df['pnlpercentage'], color=colors)
    ax.set_ylabel("Profit/Loss (%)")
    ax.set_title("PnL per Script", fontsize=14, weight="bold")
    ax.axhline(0, color='black', linewidth=0.8)
    ax.tick_params(axis='x', rotation=75, labelsize=9)
    ax.grid(axis='y', linestyle='--', alpha=0.7)

    if not df['pnlpercentage'].empty:
        pnl_data = df['pnlpercentage']
        abs_max = max(abs(pnl_data.min()), abs(pnl_data.max()))
        ax.set_ylim(-abs_max * 1.1, abs_max * 1.1)

    return fig

# ====== CSV Export Function ======

def export_portfolio_csv(df: pd.DataFrame):

    if df.empty or "Error" in df.columns or "Message" in df.columns:
        logging.warning("Export aborted: Input DataFrame contains an error, a message, or is empty.")
        return None

    try:
        export_dir = "downloadrohit"
        os.makedirs(export_dir, exist_ok=True)

        filepath = os.path.join(export_dir, f"dashboard_portfolio.csv")

        export_df = df.copy()

        # Format currency columns for better readability in the CSV
        for col in ['averageprice', 'ltp', 'profitandloss', 'invested_value', 'current_value']:
            if col in export_df.columns:
                export_df[col] = export_df[col].apply(lambda x: f'{x:,.2f}')

        export_df.to_csv(filepath, index=False)

        logging.info(f"Successfully exported portfolio to {filepath}")
        return filepath

    except Exception as e:
        logging.exception("An error occurred during CSV export.")
        return None

# ====== Master Analysis Function ======

def analyze_portfolio():

    df = get_portfolio_data()

    # Critical: Ensure a consistent number of return values for the UI
    if df.empty or "Error" in df.columns or "Message" in df.columns:
        error_msg = df.iloc[0,0] if not df.empty else "No portfolio data returned."
        # Return 6 items: df, summary, fig1, fig2, fig3, filepath
        return (
            df,
            pd.DataFrame([{"Message": error_msg}]),
            _create_placeholder_fig("Industry chart not available"),
            _create_placeholder_fig("Value chart not available"),
            _create_placeholder_fig("PnL chart not available"),
            None  # No file path on error
        )

    summary_df = calculate_metrics(df)
    fig1, fig2 = plot_charts(df)
    fig3 = plot_pnl_bars(df)

    # Generate the CSV file for download
    csv_filepath = export_portfolio_csv(df)

    # Round numerical columns for cleaner display in the UI Dataframe
    display_cols_to_round = ['averageprice', 'ltp', 'profitandloss', 'pnlpercentage', 'invested_value', 'current_value']
    for col in display_cols_to_round:
        if col in df.columns:
            # Ensure column is numeric before rounding
            df[col] = pd.to_numeric(df[col], errors='coerce').round(2)

    df = df.sort_values(by='tradingsymbol')

    return df, summary_df, fig1, fig2, fig3, csv_filepath

# **Master portfolio**

In [48]:
PORTFOLIO_FILE = master_portfolio
ARCHIVE_FILE = archive
PORTFOLIO_COLUMNS = ['date', 'symbol', 'parcel', 'quantity', 'avg_price', 'exchange', 'SWEET_Value']

In [49]:
def load_scrip_masters():
    global nse_symbol_token_map, bse_symbol_token_map, nse_symbols_list, bse_symbols_list
    exchange_map = {
        'NSE': {'file': nse, 'map': nse_symbol_token_map, 'list': nse_symbols_list},
        'BSE': {'file': bse, 'map': bse_symbol_token_map, 'list': bse_symbols_list}
    }
    for exchange, data in exchange_map.items():
        if os.path.exists(data['file']):
            logging.info(f"Loading {exchange} scrip master from local file...")
            df = pd.read_csv(data['file'])
            df['token'] = df['token'].astype(str)
            current_map = pd.Series(df.token.values, index=df.symbol).to_dict()
            data['map'].update(current_map)
            exchange_map[exchange]['list'][:] = sorted(list(current_map.keys()))
            logging.info(f"{exchange} Symbol-to-Token map created with {len(exchange_map[exchange]['list'])} symbols.")
        else:
            logging.warning(f"'{data['file']}' not found. {exchange} lookups will fail.")
    nse_symbols_list[:] = exchange_map['NSE']['list']
    bse_symbols_list[:] = exchange_map['BSE']['list']


In [50]:
def get_api_data(api_function):
    if not smartApi: return pd.DataFrame([{"Error": "Not connected to API."}])
    try:
        data = api_function()
        if data and data.get('status') and data.get('data') is not None:
            return pd.DataFrame(data['data'])
        return pd.DataFrame([{"Message": data.get('message', 'Data is empty or error occurred')}])
    except Exception as e:
        logging.error(f"Error fetching data: {e}")
        return pd.DataFrame([{"Error": str(e)}])

get_trade_book = lambda: get_api_data(smartApi.tradeBook)
get_holdings = lambda: get_api_data(smartApi.holding)

# UPDATED CODE
def get_ltp(symbol, token, exchange):
    if not smartApi: return 0
    try:
        # Try getting the token from the symbol maps first, falling back to the provided token if not found
        # This ensures we use the correct token from our loaded masters
        token_from_map = nse_symbol_token_map.get(symbol)
        if token_from_map is None:
            token_from_map = bse_symbol_token_map.get(symbol)

        # Use the token from the map if found, otherwise use the one passed into the function
        actual_token = token_from_map if token_from_map is not None else token

        # Ensure token is a string for the API call
        if actual_token is None:
             logging.warning(f"No token found for symbol {symbol} on exchange {exchange}.")
             return 0

        actual_token_str = str(actual_token)

        response = smartApi.ltpData(exchange, symbol, actual_token_str)
        if response and response.get('status') and response.get('data'):
            return response['data']['ltp']

        # Log the API response message if status is False or data is None
        if response and response.get('message'):
             logging.error(f"API Error fetching LTP for {symbol} (Token: {actual_token_str}): {response.get('message')}")
        else:
             logging.error(f"Unknown API response for LTP for {symbol} (Token: {actual_token_str}): {response}")

        return 0
    except Exception as e:
        logging.exception(f"Exception fetching LTP for {symbol} (Token: {actual_token_str if 'actual_token_str' in locals() else 'N/A'}): {e}")
        return 0

In [51]:
def load_portfolio_from_csv(file_path=PORTFOLIO_FILE):
    if not os.path.exists(file_path):
        return pd.DataFrame(columns=PORTFOLIO_COLUMNS)
    try:
        df = pd.read_csv(file_path)
        return df.sort_values(by=['symbol', 'parcel']).reset_index(drop=True)
    except Exception as e:
        logging.error(f"Error loading CSV '{file_path}': {e}")
        return pd.DataFrame(columns=PORTFOLIO_COLUMNS)

def load_archive_from_csv(file_path):
    if not os.path.exists(file_path):
        return pd.DataFrame(columns=PORTFOLIO_COLUMNS)
    try:
        df = pd.read_csv(file_path)
        return df.sort_values(by=['symbol', 'parcel']).reset_index(drop=True)
    except Exception as e:
        logging.error(f"Error loading CSV '{file_path}': {e}")
        return pd.DataFrame(columns=PORTFOLIO_COLUMNS)

def save_portfolio_to_csv(df, file_path=PORTFOLIO_FILE):
    try:
        # Ensure all required columns exist, adding them if necessary
        for col in PORTFOLIO_COLUMNS:
            if col not in df.columns:
                df[col] = None
        # Reorder columns to maintain consistency
        df[PORTFOLIO_COLUMNS].to_csv(file_path, index=False)
        logging.info(f"Portfolio saved to {file_path}")
        return f"✅ Successfully saved {len(df)} rows to {file_path}."
    except Exception as e:
        logging.error(f"Error saving portfolio: {e}")
        return f"❗️ Error saving portfolio: {e}"

def archive_entry(df_entry):
    try:
        df_entry.to_csv(ARCHIVE_FILE, mode='a', header=not os.path.exists(ARCHIVE_FILE), index=False)
        logging.info(f"Archived {len(df_entry)} entr(y/ies) to {ARCHIVE_FILE}")
    except Exception as e:
        logging.error(f"Failed to archive entry: {e}")


In [52]:
def download_holdings_as_master_portfolio():
    holdings_df = get_holdings()
    if holdings_df.empty or "Error" in holdings_df.columns or "Message" in holdings_df.columns:
        return "Could not download holdings or portfolio is empty.", pd.DataFrame()

    rows = [{
        'date': datetime.now().strftime('%Y-%m-%d'),
        'symbol': item['tradingsymbol'],
        'parcel': 'P1',
        'quantity': int(item['quantity']),
        'avg_price': float(item['averageprice']),
        'exchange': item['exchange'],
        'token': item['symboltoken']
    } for _, item in holdings_df.iterrows()]

    df_to_save = pd.DataFrame(rows)
    status = save_portfolio_to_csv(df_to_save)
    return f"Successfully created master portfolio. {status}", df_to_save

def check_mismatches():
    master_df = load_portfolio_from_csv()
    angel_df = get_holdings()
    alerts = []

    if "Error" in angel_df.columns:
        return "❌ Could not fetch Angel One holdings to check for mismatches."

    # --- NEW LOGIC TO CALCULATE NET POSITION ---
    master_summary_list = []
    # Group by stock symbol to process each one individually
    for symbol, group in master_df.groupby('symbol'):
        # Calculate total buys (parcels starting with 'P')
        total_p = group[group['parcel'].str.startswith('P')]['quantity'].sum()
        # Calculate total sells (parcels starting with 'S')
        total_s = group[group['parcel'].str.startswith('S')]['quantity'].sum()
        # Calculate the net position
        net_quantity = total_p - total_s
        master_summary_list.append({'symbol': symbol, 'master_qty': net_quantity})

    if not master_summary_list:
        master_summary = pd.DataFrame(columns=['symbol', 'master_qty'])
    else:
        master_summary = pd.DataFrame(master_summary_list)

    # Prepare Angel One holdings data
    if angel_df.empty:
         angel_summary = pd.DataFrame(columns=['symbol', 'angel_qty'])
    else:
        angel_summary = angel_df[['tradingsymbol', 'quantity']].copy()
        angel_summary['quantity'] = pd.to_numeric(angel_summary['quantity'])
        angel_summary.rename(columns={'tradingsymbol': 'symbol', 'quantity': 'angel_qty'}, inplace=True)

    # Merge the calculated net positions with the broker's data
    comparison_df = pd.merge(master_summary, angel_summary, on='symbol', how='outer').fillna(0)

    for _, row in comparison_df.iterrows():
        master_qty = int(row['master_qty'])
        angel_qty = int(row['angel_qty'])

        if master_qty != angel_qty:
            symbol = row['symbol']

            # Get the full parcel details for the mismatched stock
            stock_details_df = master_df[master_df['symbol'] == symbol]

            # Create the breakdown string (e.g., "P1: 100, P2: 180, S2: 140")
            parcel_breakdown = ", ".join([
                f"{detail_row['parcel']}: {int(detail_row['quantity'])}"
                for _, detail_row in stock_details_df.iterrows()
            ])

            if not parcel_breakdown:
                parcel_breakdown = "No holdings in master file"

            # Construct the new, correct alert message
            alert_message = (
                f"- **{symbol}**: Mismatch!\n"
                f"  - Master Portfolio Net: **{master_qty}** ({parcel_breakdown})\n"
                f"  - Angel One Holdings: **{angel_qty}**"
            )
            alerts.append(alert_message)

    if not alerts:
        return "✅ No mismatches found between Master Portfolio and Angel One holdings."

    return "🚨 **Mismatch Alert!**\n\n" + "\n\n".join(alerts)


In [53]:
def preview_trades_from_tradebook():
    trades_df = get_trade_book()
    trades_df['fillsize'] = pd.to_numeric(trades_df['fillsize'], errors='coerce')
    trades_df['fillprice'] = pd.to_numeric(trades_df['fillprice'], errors='coerce')

    if trades_df.empty or 'transactiontype' not in trades_df.columns:
        return pd.DataFrame(), "Trade book is empty or could not be fetched."

    # Load the current master portfolio to check existing parcels
    master_portfolio_df = load_portfolio_from_csv()
    # Load SWEET lookup table once
    sweet_df = load_sweet_code()

    # --- New Consolidation Logic ---
    # Group trades by symbol and transaction type first to determine parcels correctly
    consolidated_trades = []
    for (symbol, tx_type), group in trades_df.groupby(['tradingsymbol', 'transactiontype']):
        exchange = group['exchange'].iloc[0] # Assume exchange is consistent for a symbol/tx_type group

        # Determine the parcel based on transaction type and existing parcels
        parcel = None
        existing_parcels = master_portfolio_df[master_portfolio_df['symbol'] == symbol]['parcel'].tolist()

        if tx_type == 'BUY':
            if 'P1' not in existing_parcels: parcel = 'P1'
            elif 'P2' not in existing_parcels: parcel = 'P2'
            elif 'P3' not in existing_parcels: parcel = 'P3'
            elif 'P4' not in existing_parcels: parcel = 'P4'
            else: parcel = 'P1' # Default if all P parcels exist

            # Consolidate multiple BUY entries for the same parcel
            total_qty = group['fillsize'].sum()
            total_value = (group['fillsize'] * group['fillprice']).sum()
            avg_price = total_value / total_qty if total_qty > 0 else 0

            if total_qty > 0: # Only add if there's a quantity
                 sweet_value = sweet_value_for_symbol(symbol, master_portfolio_df, sweet_df) if parcel == 'P1' else None

                 consolidated_trades.append({
                    'date': datetime.now().strftime('%d-%m-%Y'), # Format date as DD-MM-YYYY
                    'symbol': symbol,
                    'parcel': parcel,
                    'quantity': int(total_qty),
                    'avg_price': round(avg_price, 2),
                    'exchange': exchange,
                    'SWEET_Value': sweet_value
                 })

        elif tx_type == 'SELL':
            # Determine SELL parcel (S1, S2, S3, S4) based on the highest existing P parcel
            if 'P4' in existing_parcels: parcel = 'S4'
            elif 'P3' in existing_parcels: parcel = 'S3'
            elif 'P2' in existing_parcels: parcel = 'S2'
            elif 'P1' in existing_parcels: parcel = 'S1'
            else: parcel = 'S1' # Default if no P parcels exist

            # For SELLs, list each trade book entry as a separate row in the preview,
            # but still determine the parcel based on the *current* master portfolio state.
            for _, trade in group.iterrows():
                 consolidated_trades.append({
                    'date': datetime.now().strftime('%d-%m-%Y'), # Format date as DD-MM-YYYY
                    'symbol': symbol,
                    'parcel': parcel, # Use the determined parcel for all sell trades in this group
                    'quantity': int(trade['fillsize']),
                    'avg_price': float(trade['fillprice']),
                    'exchange': exchange,
                    'SWEET_Value': None # SWEET_Value is only for BUY P1 entries
                 })


    preview_df = pd.DataFrame(consolidated_trades)
    return preview_df, f"Found {len(preview_df)} consolidated entries from trade book with calculated parcels."


def add_preview_to_portfolio(current_portfolio_df, preview_df):
    if preview_df.empty:
        return current_portfolio_df, "Preview is empty, nothing to add."

    # Ensure columns match before concatenating
    # Drop 'Delete' from current_portfolio_df temporarily if it exists
    portfolio_cols_no_delete = [col for col in current_portfolio_df.columns if col != 'Delete']
    current_portfolio_subset = current_portfolio_df[portfolio_cols_no_delete]

    # Ensure preview_df has all necessary columns, add missing ones with None
    for col in current_portfolio_subset.columns:
        if col not in preview_df.columns:
            preview_df[col] = None

    # Ensure columns are in the same order and have compatible dtypes
    preview_df = preview_df[current_portfolio_subset.columns]
    # Attempt to cast columns to match current_portfolio_subset
    for col in current_portfolio_subset.columns:
        try:
            preview_df[col] = preview_df[col].astype(current_portfolio_subset[col].dtype)
        except Exception as e:
            logging.warning(f"Could not cast column {col} in preview_df: {e}")


    combined_df = pd.concat([current_portfolio_subset, preview_df], ignore_index=True)

    # Re-add the 'Delete' column if it was present initially
    if 'Delete' in current_portfolio_df.columns:
        combined_df['Delete'] = False
        combined_df = combined_df[PORTFOLIO_COLUMNS + ['Delete']] # Ensure order

    return combined_df.sort_values(by=['symbol', 'parcel']).reset_index(drop=True), "✅ Entries added to portfolio editor. Please review and save."


def load_and_display_portfolio():

    df = load_portfolio_from_csv() # It uses your existing function to get the data
    df["avg_price"] = df["avg_price"].round(2)

    if not df.empty:
        df['Delete'] = False
        # Reorder columns to have 'Delete' at the end for consistency
        df = df[PORTFOLIO_COLUMNS + ['Delete']]
    else:
         # If file doesn't exist, return an empty DataFrame with all columns
        df = pd.DataFrame(columns=PORTFOLIO_COLUMNS + ['Delete'])

    return df, f"Loaded {len(df)} rows from {PORTFOLIO_FILE}."

def delete_and_save_rows(df):

    if 'Delete' not in df.columns or df['Delete'].sum() == 0:
        return df, "No rows were selected for deletion."

    rows_to_delete_count = int(df['Delete'].sum())

    # Create a new dataframe keeping only the rows where 'Delete' is False
    cleaned_df = df[df['Delete'] == False].copy()

    # Drop the temporary 'Delete' column before saving
    cleaned_df.drop(columns=['Delete'], inplace=True)

    # Save the cleaned dataframe to the CSV file
    save_status = save_portfolio_to_csv(cleaned_df)

    # Prepare the dataframe to be displayed back in the UI (with the Delete column)
    if not cleaned_df.empty:
        cleaned_df['Delete'] = False

    status_message = f"✅ Deleted {rows_to_delete_count} row(s). {save_status}"

    return cleaned_df, status_message

In [54]:
def archive_matched_trades(portfolio_df):
    """Processes the portfolio DataFrame to archive matched trades."""
    if portfolio_df is None or portfolio_df.empty:
        return "No portfolio data to process."

    df = portfolio_df.copy()
    df.to_csv("downloadrohit/backup/master_portfolio.csv", index=False)
    df['quantity'] = pd.to_numeric(df['quantity'], errors='coerce').fillna(0)
    df['parcel_type'] = df['parcel'].str[0]
    df['parcel_num'] = df['parcel'].str[1:]

    archive_temp_df = pd.read_csv(archive)
    archive_temp_df.to_csv("downloadrohit/backup/archive.csv", index=False)

    ps_parcels = df[df['parcel_type'].isin(['P', 'S']) & df['parcel_num'].str.isdigit()].copy()
    other_parcels = df[~df.index.isin(ps_parcels.index)]

    if not ps_parcels.empty:
        ps_parcels['signed_qty'] = np.where(ps_parcels['parcel_type'] == 'P', ps_parcels['quantity'], -ps_parcels['quantity'])
        ps_parcels['net_qty'] = ps_parcels.groupby(['symbol', 'parcel_num'])['signed_qty'].transform('sum')
        is_archivable = np.isclose(ps_parcels['net_qty'], 0)
        rows_to_archive_df = ps_parcels[is_archivable]
        rows_to_keep_ps = ps_parcels[~is_archivable]
    else:
        rows_to_archive_df = pd.DataFrame()
        rows_to_keep_ps = pd.DataFrame()

    updated_portfolio_df = pd.concat([rows_to_keep_ps, other_parcels])

    cols_to_drop = ['parcel_type', 'parcel_num', 'signed_qty', 'net_qty']
    rows_to_archive_df = rows_to_archive_df.drop(columns=cols_to_drop, errors='ignore')
    updated_portfolio_df = updated_portfolio_df.drop(columns=cols_to_drop, errors='ignore')

    archived_count = 0
    if not rows_to_archive_df.empty:
        archive_entry(rows_to_archive_df[PORTFOLIO_COLUMNS])
        archived_count = len(rows_to_archive_df)

    if not updated_portfolio_df.empty:
        updated_portfolio_df[PORTFOLIO_COLUMNS].to_csv(PORTFOLIO_FILE, index=False)
    else:
        pd.DataFrame(columns=PORTFOLIO_COLUMNS).to_csv(PORTFOLIO_FILE, index=False)

    return f"✅ Process complete! Archived {archived_count} rows. The portfolio file has been updated."

# --- Wrapper Function for Gradio ---
def run_archiving_process():
    """
    This function acts as a bridge between the Gradio button and your main script.
    It handles file loading and error checking.
    """
    try:
        yield "Processing... Please wait." # Gives immediate feedback
        time.sleep(SLEEP*5) # Small delay to make the "Processing" message visible
        df = pd.read_csv(PORTFOLIO_FILE)
        status_message = archive_matched_trades(df)
        yield status_message
    except FileNotFoundError:
        yield f"❌ ERROR: The file '{PORTFOLIO_FILE}' was not found. Please make sure it's in the same directory."
    except Exception as e:
        yield f"❌ An unexpected error occurred: {e}"


    # archive_button = gr.Button("🚀 Run Archiving Process", variant="primary")

    # # Connect the button click to the wrapper function
    # archive_button.click(
    #     fn=run_archiving_process,
    #     inputs=None,
    #     outputs=status_textbox
    # )


# **Automated Buy and Sell**

In [55]:
# ---------- Helper functions ----------
def _parse_date_safe(d):
    """Parse a date in DD-MM-YYYY or ISO format robustly. Return datetime.date or None."""
    if pd.isna(d):
        return None
    if isinstance(d, datetime):
        return d.date()
    s = str(d).strip()
    for fmt in ("%d-%m-%Y", "%Y-%m-%d", "%d/%m/%Y", "%d-%b-%Y"):
        try:
            return datetime.strptime(s, fmt).date()
        except Exception:
            continue
    # fallback: try pandas
    try:
        return pd.to_datetime(s, dayfirst=True).date()
    except Exception:
        logging.warning(f"Could not parse date: {d}")
        return None

def days_since(date_obj):
    """Return integer days since date_obj (date or string)."""
    if date_obj is None:
        return None
    if not isinstance(date_obj, (datetime, )):
        d = _parse_date_safe(date_obj)
    else:
        d = date_obj.date()
    if d is None:
        return None
    return (datetime.now().date() - d).days

def compute_pct_change(from_price, to_price):
    """Return percent change (to_price - from_price)/from_price * 100; safe on zero."""
    try:
        if from_price == 0 or pd.isna(from_price):
            return None
        return ((to_price - from_price) / float(from_price)) * 100.0
    except Exception:
        return None

# ---------- Read rules (backward compatible) ----------
def read_rules(file_path=rule_book):
    """
    Reads rule_book CSV and returns:
      - a dict mapping variable_name -> numeric 'value' (backwards compatible),
      - stores raw df under key '_df' for richer lookup,
      - stores symbol overrides under key '_overrides' (dict mapping symbol->{variable_name: row_dict})
    Old callers that used rules.get('S1', default) will continue to work.
    """
    try:
        df = pd.read_csv(file_path, dtype=str).fillna('')
        # ensure numeric fields coerced where relevant
        numeric_cols = ['value', 'DaysDiff', 'Quantity']
        for c in numeric_cols:
            if c in df.columns:
                df[c] = pd.to_numeric(df[c], errors='coerce')

        # Build basic mapping: variable_name -> value (numeric) for backward compatibility
        var_to_val = {}
        for _, r in df.iterrows():
            vn = r.get('variable_name', '')
            if not vn:
                continue
            v = r.get('value')
            try:
                var_to_val[vn] = float(v) if not pd.isna(v) else None
            except Exception:
                var_to_val[vn] = None

        # Build overrides mapping: Exception (symbol) -> {variable_name: row_dict}
        overrides = {}
        for _, r in df.iterrows():
            exc = str(r.get('Exception', '')).strip()
            if exc == '' or pd.isna(exc):
                continue
            if exc not in overrides:
                overrides[exc] = {}
            overrides[exc][r['variable_name']] = r.to_dict()

        # Keep full df for lookups
        var_to_val['_df'] = df
        var_to_val['_overrides'] = overrides
        return var_to_val

    except Exception as e:
        logging.error(f"Error reading '{file_path}': {e}. Using defaults.")
        # Provide minimal defaults to maintain prior behavior
        defaults = {
            'P2': 5, 'P3': 10, 'P4': 20,
            'S1': 30, 'S1Day': 15, 'SWEET_Diff': 5,
            'S2': 100, 'S3': 100, 'S4': 100,
            '_df': pd.DataFrame(), '_overrides': {}
        }
        return defaults

def get_rule(rules, symbol, variable_name):
    """
    Return a dict-like row for the requested rule.
    If an override exists for the symbol (rules['_overrides']), use it.
    Otherwise, return the generic row from rules['_df'] where variable_name matches.
    The returned object is a dict with keys like 'value', 'DaysDiff', 'Quantity' (floats if present).
    If not found, returns a default dict with 'value' from rules.get(variable_name).
    """
    try:
        overrides = rules.get('_overrides', {})
        df = rules.get('_df', pd.DataFrame())
        if symbol in overrides and variable_name in overrides[symbol]:
            row = overrides[symbol][variable_name]
            # normalize numeric fields
            for k in ['value', 'DaysDiff', 'Quantity']:
                if k in row and (row[k] is not None and row[k] != ''):
                    try:
                        row[k] = float(row[k])
                    except Exception:
                        row[k] = None
            return row

        # find generic
        if isinstance(df, pd.DataFrame) and not df.empty:
            matched = df[df['variable_name'] == variable_name]
            if not matched.empty:
                r = matched.iloc[0].to_dict()
                for k in ['value', 'DaysDiff', 'Quantity']:
                    if k in r and (r[k] is not None and r[k] != ''):
                        try:
                            r[k] = float(r[k])
                        except Exception:
                            r[k] = None
                return r

        # fallback: return numeric value and None for others
        return {
            'variable_name': variable_name,
            'value': rules.get(variable_name),
            'DaysDiff': None,
            'Quantity': None,
            'Exception': None
        }
    except Exception as e:
        logging.error(f"get_rule error: {e}")
        return {'variable_name': variable_name, 'value': rules.get(variable_name), 'DaysDiff': None, 'Quantity': None}

def load_sweet_code(file_path=SWEET_Code_path):
    """Load SWEET_Code csv to map codes to SWEET_Value. Returns DataFrame."""
    try:
        df = pd.read_csv(file_path, dtype=str).fillna('')
        if 'SWEET_Value' in df.columns:
            df['SWEET_Value'] = pd.to_numeric(df['SWEET_Value'], errors='coerce')
        return df
    except Exception as e:
        logging.warning(f"Unable to load SWEET_Code: {e}")
        return pd.DataFrame()

def sweet_value_for_symbol(symbol, master_df=None, sweet_df=None):
    """
    Get current SWEET_Value for a symbol:
      - Prefer latest entry in master_portfolio for that symbol (if present)
      - Else try lookup in sweet_df by matching NSE_Code or BSE_Code
      - Else return None
    Handles '-EQ' suffix difference in NSE_Code.
    """
    try:
        if master_df is not None and not master_df.empty:
            srows = master_df[master_df['symbol'] == symbol]
            if not srows.empty:
                # take last non-null SWEET_Value from master_portfolio rows for symbol
                vals = srows['SWEET_Value'].dropna().values
                if len(vals) > 0:
                    return float(vals[-1])
        if sweet_df is None:
            sweet_df = load_sweet_code()
        if not sweet_df.empty:
            # try direct match in NSE_Code/BSE_Code columns
            for col in ['NSE_Code', 'BSE_Code']: # Only check NSE_Code and BSE_Code in SWEET_Code.csv
                if col in sweet_df.columns:
                    # Remove '-EQ' from symbol if present for comparison with NSE_Code
                    lookup_symbol = symbol.replace('-EQ', '') if col == 'NSE_Code' else symbol
                    m = sweet_df[sweet_df[col] == lookup_symbol]
                    if not m.empty and 'SWEET_Value' in m.columns:
                        v = m.iloc[0]['SWEET_Value']
                        if not pd.isna(v):
                            return float(v)
        return None
    except Exception as e:
        logging.error(f"sweet_value_for_symbol error: {e}")
        return None
def current_sweet_value_for_symbol(symbol, sweet_df=None):
    """
    Get current SWEET_Value for a symbol:
      try lookup in sweet_df by matching NSE_Code or BSE_Code
      - Else return None
    Handles '-EQ' suffix difference in NSE_Code.
    """
    try:
        if sweet_df is None:
            sweet_df = load_sweet_code()
        if not sweet_df.empty:
            # try direct match in NSE_Code/BSE_Code columns
            for col in ['NSE_Code', 'BSE_Code']: # Only check NSE_Code and BSE_Code in SWEET_Code.csv
                if col in sweet_df.columns:
                    # Remove '-EQ' from symbol if present for comparison with NSE_Code
                    lookup_symbol = symbol.replace('-EQ', '') if col == 'NSE_Code' else symbol
                    m = sweet_df[sweet_df[col] == lookup_symbol]
                    if not m.empty and 'SWEET_Value' in m.columns:
                        v = m.iloc[0]['SWEET_Value']
                        if not pd.isna(v):
                            return float(v)
        return None
    except Exception as e:
        logging.error(f"sweet_value_for_symbol error: {e}")
        return None

def current_active_parcels(master_df, symbol):
    """
    Return dict of active parcel quantities and the first-row details for each parcel present for the symbol.
    This function is defensive: sums quantities for each parcel and subtracts sell parcels S1..S4 to compute net.
    Returns:
      {
        'quantities': { 'P1': net_qty, 'P2': net_qty, ... },
        'rows': { 'P1': row_series_of_first_P1, ... }  # first matching row for avg_price/date/token/exchange
      }
    """
    if master_df is None or master_df.empty:
        return {'quantities': {}, 'rows': {}}
    grp = master_df[master_df['symbol'] == symbol]
    if grp.empty:
        return {'quantities': {}, 'rows': {}}

    # sum up quantities for each parcel label
    qtys = {}
    rows = {}
    for _, r in grp.iterrows():
        parcel = r.get('parcel', '')
        try:
            q = float(r.get('quantity', 0))
        except Exception:
            q = 0.0
        qtys[parcel] = qtys.get(parcel, 0.0) + q
        # store first row details for parcel if not stored
        if parcel.startswith('P') and parcel not in rows:
            rows[parcel] = r
        # But also store S rows maybe used to reduce net
        if parcel.startswith('S') and parcel not in rows:
            rows[parcel] = r

    # compute net: for each P* compute net = Pn - Sn (if Sn exists)
    net = {}
    for n in ['P1', 'P2', 'P3', 'P4']:
        pqty = qtys.get(n, 0.0)
        s_label = 'S' + n[1]
        sqty = qtys.get(s_label, 0.0)
        net_qty = pqty - sqty
        # Defensive: if both P4 and S4 exist, treat present status as P3 only (following your rule).
        # That means if P4>0 and S4>0 then logically we should treat it as matched; so set net for P4 to 0
        if n == 'P4' and qtys.get('P4', 0) > 0 and qtys.get('S4', 0) > 0:
            # treat as if P4 matched -> present status should be P3
            net_qty = 0.0
        net[n] = net_qty

    return {'quantities': net, 'rows': rows}

def append_trade_row(master_df, symbol, parcel, quantity, price, exchange, date=None, sweet_value=None):
    """
    Append a trade row (buy or sell) to master_df and return the new DataFrame.
    Columns preserved: date, symbol, parcel, quantity, avg_price, exchange, SWEET_Value, token (if present)
    """
    if date is None:
        date = datetime.now().strftime("%d-%m-%Y")
    new_row = {
        'date': date,
        'symbol': symbol,
        'parcel': parcel,
        'quantity': int(math.floor(quantity)) if quantity is not None else 0,
        'avg_price': float(price) if price is not None else 0.0,
        'exchange': exchange if exchange is not None else '',
        'SWEET_Value': sweet_value if sweet_value is not None else None,
    }
    # Attempt to preserve token column if present in master_df schema (set None by default)
    if 'token' in master_df.columns:
        new_row['token'] = None
    # Append
    appended = master_df.copy()
    appended = appended.append(new_row, ignore_index=True)
    logging.info(f"append_trade_row: appended {new_row}")
    return appended

# ---------- Main suggestion logic (refactored) ----------
def generate_suggestions(master_portfolio_df, rules, dry_run=True):
    """
    master_portfolio_df: DataFrame of master_portfolio containing P1/P2/P3/P4/S1.. entries.
    rules: output of read_rules() (backwards-compatible dict with '_df' and '_overrides').
    dry_run: if True -> do not persist changes; function only returns suggestions list (but still simulates calculations).
    Returns: suggestions_df (DataFrame)
    Each suggestion row: Select (bool), Action (BUY/SELL), Symbol, Reason, Quantity, Price, Exchange, Parcel
    """
    suggestions = []
    if master_portfolio_df is None or master_portfolio_df.empty:
        return pd.DataFrame()

    # Load SWEET lookup table once
    sweet_df = load_sweet_code()

    # to download all the suggestions generated in rohit_audit.csv.
    rohit_audit=[]


    # Group by symbol
    for symbol, group in master_portfolio_df.groupby('symbol'):
        current_quantity=0
        sell_qty=0
        buy_qty=0
        threshold_s1day=0
        sweet_pct_change=0
        s1day_days=0
        p2_threshold=0
        p3_threshold=0
        p2_qty=0
        p4_threshold=0
        p2_price=0
        p3_qty=0
        p3_price=0
        p4_qty=0

        try:
            # Build active parcels state
            state = current_active_parcels(master_portfolio_df, symbol)
            net_qty = state['quantities']  # net quantities per P1..P4 after S* adjustments
            rows = state['rows']  # representative rows per P parcel
            # Need a token and exchange to call get_ltp - prefer the P1 row then other P rows
            token = None
            exchange = None
            for p in ['P1', 'P2', 'P3', 'P4']:
                r = rows.get(p)
                if r is not None:
                    token = r.get('token', None) if 'token' in r else None
                    exchange = r.get('exchange', None) if 'exchange' in r else None
                    break

            # Determine present top-level status: highest Pn with net > 0 (P4 > P3 > P2 > P1)
            present_status = None
            for p in ['P4', 'P3', 'P2', 'P1']:
                if net_qty.get(p, 0) > 0:
                    present_status = p
                    break

            # LTP retrieval - rely on existing get_ltp function in codebase
            # This function signature used elsewhere: get_ltp(symbol, token, exchange)
            try:
                ltp = get_ltp(symbol, token, exchange) if 'get_ltp' in globals() else None
            except Exception:
                ltp = None
            if ltp is None:
                # fallback: if group has avg_price (last recorded price), use it
                ltp = rows.get(present_status, {}).get('avg_price') if present_status else None
            if ltp is None or ltp == 0:
                # Can't make decisions without LTP
                continue

            # P1 must exist for almost all comparisons
            if 'P1' not in rows or rows['P1'] is None:
                # nothing to do if we don't have original P1
                continue

            p1_row = rows['P1']
            p1_price = float(p1_row.get('avg_price', 0.0))
            p1_qty_total = float(master_portfolio_df[(master_portfolio_df['symbol'] == symbol) & (master_portfolio_df['parcel'] == 'P1')]['quantity'].sum())
            p1_date = p1_row.get('date', None)
            p1_days = days_since(p1_date)

            # current SWEET values (P1 stored SWEET if present)
            p1_sweet = None
            try:
                if 'SWEET_Value' in p1_row and not pd.isna(p1_row.get('SWEET_Value')):
                    p1_sweet = float(p1_row.get('SWEET_Value'))
            except Exception:
                p1_sweet = None
            current_sweet = current_sweet_value_for_symbol(symbol, sweet_df)
            # if current_sweet is None, fallback to p1_sweet
            if current_sweet is None:
                current_sweet = p1_sweet

            # --- Profit branch checks for P1 (S1 and S1Day) ---
            # S1
            s1_rule = get_rule(rules, symbol, 'S1')
            s1_value = float(s1_rule.get('value')) if s1_rule.get('value') is not None else None
            s1_qty_pct = float(s1_rule.get('Quantity')) if s1_rule.get('Quantity') is not None else None

            if present_status == 'P1' and s1_value is not None and s1_qty_pct is not None:
                threshold_s1 = p1_price * (1.0 + (s1_value / 100.0))
                if ltp >= threshold_s1:
                    # sell S1: sell s1_qty_pct % of current_quantity
                    # current_quantity is net of P1 minus S1
                    current_quantity = net_qty.get('P1', 0)
                    sell_qty = math.floor(current_quantity * (s1_qty_pct / 100.0))
                    if sell_qty < 1 and current_quantity >= 1:
                        sell_qty = 1
                    if sell_qty > 0:
                        suggestions.append({
                            'Select': False, 'Action': 'SELL', 'Symbol': symbol,
                            'Reason': f"S1 Target Met: LTP {ltp:.2f} >= {threshold_s1:.2f}",
                            'Quantity': int(sell_qty), 'Price': round(ltp, 2),
                            'Exchange': exchange, 'Parcel': 'S1'
                        })
                        rohit_audit.append({"symbol": symbol,"buy_qty":buy_qty,"net_qty": net_qty,"ltp": ltp,"p1_price": p1_price,"p1_qty_total": p1_qty_total,"p1_date": p1_date,"p1_days": p1_days,"p1_sweet": p1_sweet,"current_sweet": current_sweet,"threshold_s1": threshold_s1,"current_quantity": current_quantity,"sell_qty": sell_qty,"threshold_s1day": threshold_s1day,"sweet_pct_change": sweet_pct_change,"s1day_days": s1day_days,"p2_threshold": p2_threshold,"p3_threshold": p3_threshold,"p2_qty": p2_qty,"p4_threshold": p4_threshold,"p2_price": p2_price,"p3_qty": p3_qty,"p3_price": p3_price,"p4_qty": p4_qty          })

                        # For P1 profit branch, after a sell suggestion, stop further suggestions for that symbol
                        # to maintain priority
                        continue

                # S1Day: discount smaller gain but SWEET decreased and within DaysDiff
                s1day_rule = get_rule(rules, symbol, 'S1Day')
                s1day_value = s1day_rule.get('value')
                s1day_days = s1day_rule.get('DaysDiff')
                s1day_qty_pct = s1day_rule.get('Quantity')

                sweet_diff_rule = get_rule(rules, symbol, 'SWEET_Diff')
                sweet_diff_pct = sweet_diff_rule.get('value') if sweet_diff_rule else None

                if s1day_value is not None and s1day_qty_pct is not None and s1day_days is not None and p1_days is not None:
                    threshold_s1day = p1_price * (1.0 + (float(s1day_value) / 100.0))
                    # SWEET change percent (current from P1)
                    if p1_sweet is not None and current_sweet is not None:
                        sweet_pct_change = compute_pct_change(p1_sweet, current_sweet)
                    else:
                        sweet_pct_change = None
                    # Condition: LTP >= threshold_s1day and purchase age <= DaysDiff and SWEET decreased by >= SWEET_Diff
                    if (ltp >= threshold_s1day) and (p1_days <= int(s1day_days)):
                        if (sweet_pct_change is not None) and (sweet_diff_pct is not None):
                            if sweet_pct_change <= -abs(float(sweet_diff_pct)):
                                # sell
                                current_quantity = net_qty.get('P1', 0)
                                sell_qty = math.floor(current_quantity * (s1day_qty_pct / 100.0))
                                if sell_qty < 1 and current_quantity >= 1:
                                    sell_qty = 1
                                if sell_qty > 0:
                                    suggestions.append({
                                        'Select': False, 'Action': 'SELL', 'Symbol': symbol,
                                        'Reason': f"S1Day Target Met: LTP {ltp:.2f} >= {threshold_s1day:.2f} & SWEET down {abs(sweet_pct_change):.2f}%",
                                        'Quantity': int(sell_qty), 'Price': round(ltp, 2),
                                        'Exchange': exchange, 'Parcel': 'S1Day'
                                    })
                                    rohit_audit.append({"symbol": symbol,"buy_qty":buy_qty,"net_qty": net_qty,"ltp": ltp,"p1_price": p1_price,"p1_qty_total": p1_qty_total,"p1_date": p1_date,"p1_days": p1_days,"p1_sweet": p1_sweet,"current_sweet": current_sweet,"threshold_s1": threshold_s1,"current_quantity": current_quantity,"sell_qty": sell_qty,"threshold_s1day": threshold_s1day,"sweet_pct_change": sweet_pct_change,"s1day_days": s1day_days,"p2_threshold": p2_threshold,"p3_threshold": p3_threshold,"p2_qty": p2_qty,"p4_threshold": p4_threshold,"p2_price": p2_price,"p3_qty": p3_qty,"p3_price": p3_price,"p4_qty": p4_qty          })

                                    continue

            # --- Loss branch / dip buys when P1 exists (P2,P3,P4) ---
            if p1_price is None or p1_price == 0:
                # cannot compute buy thresholds
                continue

            # If present_status is P1 (no P2..P4 active), check P2
            if present_status == 'P1':
                p2_rule = get_rule(rules, symbol, 'P2')
                if p2_rule.get('value') is not None and p2_rule.get('Quantity') is not None:
                    p2_threshold = p1_price * (1.0 - (float(p2_rule['value']) / 100.0))
                    if ltp <= p2_threshold:
                        # buy P2: Quantity % of P1
                        buy_qty = math.floor(p1_qty_total * (float(p2_rule['Quantity']) / 100.0))
                        if buy_qty < 1:
                            buy_qty = 1
                        suggestions.append({
                            'Select': False, 'Action': 'BUY', 'Symbol': symbol,
                            'Reason': f"P2 Target Met: LTP {ltp:.2f} <= {p2_threshold:.2f}",
                            'Quantity': int(buy_qty), 'Price': round(ltp, 2),
                            'Exchange': exchange, 'Parcel': 'P2'
                        })
                        rohit_audit.append({"symbol": symbol,"buy_qty":buy_qty,"net_qty": net_qty,"ltp": ltp,"p1_price": p1_price,"p1_qty_total": p1_qty_total,"p1_date": p1_date,"p1_days": p1_days,"p1_sweet": p1_sweet,"current_sweet": current_sweet,"threshold_s1": threshold_s1,"current_quantity": current_quantity,"sell_qty": sell_qty,"threshold_s1day": threshold_s1day,"sweet_pct_change": sweet_pct_change,"s1day_days": s1day_days,"p2_threshold": p2_threshold,"p3_threshold": p3_threshold,"p2_qty": p2_qty,"p4_threshold": p4_threshold,"p2_price": p2_price,"p3_qty": p3_qty,"p3_price": p3_price,"p4_qty": p4_qty          })

                        # Only first applied buy per symbol
                        continue

            # If present_status == P2 (P2 exists), check for P3 or S2
            if present_status == 'P2':
                p3_rule = get_rule(rules, symbol, 'P3')
                if p3_rule.get('value') is not None and p3_rule.get('Quantity') is not None:
                    p3_threshold = p1_price * (1.0 - (float(p3_rule['value']) / 100.0))
                    if ltp <= p3_threshold and 'P3' not in rows:
                        buy_qty = math.floor(p1_qty_total * (float(p3_rule['Quantity']) / 100.0))
                        if buy_qty < 1:
                            buy_qty = 1
                        suggestions.append({
                            'Select': False, 'Action': 'BUY', 'Symbol': symbol,
                            'Reason': f"P3 Target Met: LTP {ltp:.2f} <= {p3_threshold:.2f}",
                            'Quantity': int(buy_qty), 'Price': round(ltp, 2),
                            'Exchange': exchange, 'Parcel': 'P3'
                        })
                        rohit_audit.append({"symbol": symbol,"buy_qty":buy_qty,"net_qty": net_qty,"ltp": ltp,"p1_price": p1_price,"p1_qty_total": p1_qty_total,"p1_date": p1_date,"p1_days": p1_days,"p1_sweet": p1_sweet,"current_sweet": current_sweet,"threshold_s1": threshold_s1,"current_quantity": current_quantity,"sell_qty": sell_qty,"threshold_s1day": threshold_s1day,"sweet_pct_change": sweet_pct_change,"s1day_days": s1day_days,"p2_threshold": p2_threshold,"p3_threshold": p3_threshold,"p2_qty": p2_qty,"p4_threshold": p4_threshold,"p2_price": p2_price,"p3_qty": p3_qty,"p3_price": p3_price,"p4_qty": p4_qty          })

                        continue
                # S2: if LTP >= P1_price then sell all P2
                if ltp >= p1_price:
                    p2_qty = net_qty.get('P2', 0)
                    if p2_qty > 0:
                        suggestions.append({
                            'Select': False, 'Action': 'SELL', 'Symbol': symbol,
                            'Reason': f"S2 Condition Met: LTP {ltp:.2f} >= P1_price {p1_price:.2f}",
                            'Quantity': int(math.floor(p2_qty)), 'Price': round(ltp, 2),
                            'Exchange': exchange, 'Parcel': 'S2'
                        })
                        rohit_audit.append({"symbol": symbol,"buy_qty":buy_qty,"net_qty": net_qty,"ltp": ltp,"p1_price": p1_price,"p1_qty_total": p1_qty_total,"p1_date": p1_date,"p1_days": p1_days,"p1_sweet": p1_sweet,"current_sweet": current_sweet,"threshold_s1": threshold_s1,"current_quantity": current_quantity,"sell_qty": sell_qty,"threshold_s1day": threshold_s1day,"sweet_pct_change": sweet_pct_change,"s1day_days": s1day_days,"p2_threshold": p2_threshold,"p3_threshold": p3_threshold,"p2_qty": p2_qty,"p4_threshold": p4_threshold,"p2_price": p2_price,"p3_qty": p3_qty,"p3_price": p3_price,"p4_qty": p4_qty          })

                        continue

            # If present_status == P3, check for P4 or S3
            if present_status == 'P3':
                p4_rule = get_rule(rules, symbol, 'P4')
                if p4_rule.get('value') is not None and p4_rule.get('Quantity') is not None:
                    p4_threshold = p1_price * (1.0 - (float(p4_rule['value']) / 100.0))
                    if ltp <= p4_threshold and 'P4' not in rows:
                        buy_qty = math.floor(p1_qty_total * (float(p4_rule['Quantity']) / 100.0))
                        if buy_qty < 1:
                            buy_qty = 1
                        suggestions.append({
                            'Select': False, 'Action': 'BUY', 'Symbol': symbol,
                            'Reason': f"P4 Target Met: LTP {ltp:.2f} <= {p4_threshold:.2f}",
                            'Quantity': int(buy_qty), 'Price': round(ltp, 2),
                            'Exchange': exchange, 'Parcel': 'P4'
                        })
                        rohit_audit.append({"symbol": symbol,"buy_qty":buy_qty,"net_qty": net_qty,"ltp": ltp,"p1_price": p1_price,"p1_qty_total": p1_qty_total,"p1_date": p1_date,"p1_days": p1_days,"p1_sweet": p1_sweet,"current_sweet": current_sweet,"threshold_s1": threshold_s1,"current_quantity": current_quantity,"sell_qty": sell_qty,"threshold_s1day": threshold_s1day,"sweet_pct_change": sweet_pct_change,"s1day_days": s1day_days,"p2_threshold": p2_threshold,"p3_threshold": p3_threshold,"p2_qty": p2_qty,"p4_threshold": p4_threshold,"p2_price": p2_price,"p3_qty": p3_qty,"p3_price": p3_price,"p4_qty": p4_qty          })

                        continue
                # S3: if LTP >= P2_price then sell all P3
                p2_row = rows.get('P2', None)
                if p2_row is not None:
                    p2_price = float(p2_row.get('avg_price', 0.0))
                    if p2_price and ltp >= p2_price:
                        p3_qty = net_qty.get('P3', 0)
                        if p3_qty > 0:
                            suggestions.append({
                                'Select': False, 'Action': 'SELL', 'Symbol': symbol,
                                'Reason': f"S3 Condition Met: LTP {ltp:.2f} >= P2_price {p2_price:.2f}",
                                'Quantity': int(math.floor(p3_qty)), 'Price': round(ltp, 2),
                                'Exchange': exchange, 'Parcel': 'S3'
                            })
                            rohit_audit.append({"symbol": symbol,"buy_qty":buy_qty,"net_qty": net_qty,"ltp": ltp,"p1_price": p1_price,"p1_qty_total": p1_qty_total,"p1_date": p1_date,"p1_days": p1_days,"p1_sweet": p1_sweet,"current_sweet": current_sweet,"threshold_s1": threshold_s1,"current_quantity": current_quantity,"sell_qty": sell_qty,"threshold_s1day": threshold_s1day,"sweet_pct_change": sweet_pct_change,"s1day_days": s1day_days,"p2_threshold": p2_threshold,"p3_threshold": p3_threshold,"p2_qty": p2_qty,"p4_threshold": p4_threshold,"p2_price": p2_price,"p3_qty": p3_qty,"p3_price": p3_price,"p4_qty": p4_qty          })

                            continue

            # If present_status == P4, check S4 condition
            if present_status == 'P4':
                p3_row = rows.get('P3', None)
                if p3_row is not None:
                    p3_price = float(p3_row.get('avg_price', 0.0))
                    if p3_price and ltp >= p3_price:
                        p4_qty = net_qty.get('P4', 0)
                        if p4_qty > 0:
                            suggestions.append({
                                'Select': False, 'Action': 'SELL', 'Symbol': symbol,
                                'Reason': f"S4 Condition Met: LTP {ltp:.2f} >= P3_price {p3_price:.2f}",
                                'Quantity': int(math.floor(p4_qty)), 'Price': round(ltp, 2),
                                'Exchange': exchange, 'Parcel': 'S4'
                            })
                            rohit_audit.append({"symbol": symbol,"buy_qty":buy_qty,"net_qty": net_qty,"ltp": ltp,"p1_price": p1_price,"p1_qty_total": p1_qty_total,"p1_date": p1_date,"p1_days": p1_days,"p1_sweet": p1_sweet,"current_sweet": current_sweet,"threshold_s1": threshold_s1,"current_quantity": current_quantity,"sell_qty": sell_qty,"threshold_s1day": threshold_s1day,"sweet_pct_change": sweet_pct_change,"s1day_days": s1day_days,"p2_threshold": p2_threshold,"p3_threshold": p3_threshold,"p2_qty": p2_qty,"p4_threshold": p4_threshold,"p2_price": p2_price,"p3_qty": p3_qty,"p3_price": p3_price,"p4_qty": p4_qty          })

                            continue
                # If LTP declines further from P4 price: do nothing per rules

            rohit_audit.append({"symbol": symbol,"buy_qty":buy_qty,"net_qty": net_qty,"ltp": ltp,"p1_price": p1_price,"p1_qty_total": p1_qty_total,"p1_date": p1_date,"p1_days": p1_days,"p1_sweet": p1_sweet,"current_sweet": current_sweet,"threshold_s1": threshold_s1,"current_quantity": current_quantity,"sell_qty": sell_qty,"threshold_s1day": threshold_s1day,"sweet_pct_change": sweet_pct_change,"s1day_days": s1day_days,"p2_threshold": p2_threshold,"p3_threshold": p3_threshold,"p2_qty": p2_qty,"p4_threshold": p4_threshold,"p2_price": p2_price,"p3_qty": p3_qty,"p3_price": p3_price,"p4_qty": p4_qty          })

        except Exception as e:
            logging.exception(f"Error processing symbol {symbol}: {e}")
            continue

    rohit_audit_df = pd.DataFrame(rohit_audit)
    rohit_audit_df.to_csv("downloadrohit/rohit_audit_csv.csv",index=False)

    # convert suggestions to DataFrame and return
    suggestions_df = pd.DataFrame(suggestions)
    # default Select to False column if missing
    if not suggestions_df.empty and 'Select' not in suggestions_df.columns:
        suggestions_df['Select'] = False
    return suggestions_df

# ---------- Simple update_and_archive_portfolio wrapper ----------
def update_and_archive_portfolio(portfolio_df, trades_df, rules, dry_run=True):
    """
    This function orchestrates reading current master_portfolio (portfolio_df is expected to be the master_portfolio),
    generating suggestions, and optionally appending trade rows for simulated executed suggestions (if not dry_run).
    Returns: (updated_master_portfolio_df, status_string)
    NOTE: This wrapper intentionally keeps persistence calls outside; original callers manage saving to CSV.
    """
    try:
        if portfolio_df is None:
            portfolio_df = pd.DataFrame()
        suggestions_df = generate_suggestions(portfolio_df, rules, dry_run=True)

        status_msgs = []
        status_msgs.append(f"Scanned {len(portfolio_df['symbol'].unique()) if not portfolio_df.empty else 0} symbols.")
        status_msgs.append(f"Generated {len(suggestions_df)} suggestions (dry_run={dry_run}).")

        # If not dry_run, simulate executing suggestions by appending to master_portfolio
        updated = portfolio_df.copy()
        if (not dry_run) and (not suggestions_df.empty):
            for _, s in suggestions_df.iterrows():
                # map Action/Parcel into appended row
                parcel = s.get('Parcel')
                qty = s.get('Quantity', 0)
                price = s.get('Price', None)
                exch = s.get('Exchange', None)
                symbol = s.get('Symbol')
                # compute sweet_value at time of trade
                sv = sweet_value_for_symbol(symbol, updated)
                updated = append_trade_row(updated, symbol, parcel, qty, price, exch, date=datetime.now().strftime("%d-%m-%Y"), sweet_value=sv)
            status_msgs.append(f"Appended {len(suggestions_df)} trade rows to master_portfolio.")
        return updated, "\n".join(status_msgs)
    except Exception as e:
        logging.exception(f"update_and_archive_portfolio error: {e}")
        return portfolio_df, f"Error: {e}"

def on_analyze_and_suggest_click():
    rules = read_rules()
    portfolio_df = load_portfolio_from_csv()
    trades_df = get_trade_book()

    updated_portfolio_df, status = update_and_archive_portfolio(portfolio_df, trades_df, rules)

    if not updated_portfolio_df.empty:
        save_portfolio_to_csv(updated_portfolio_df)

    suggestions_df = generate_suggestions(updated_portfolio_df, rules)
    status += f"\nFound {len(suggestions_df)} new suggestions."
    return status, suggestions_df, gr.update(value=updated_portfolio_df)


def execute_selected_trades(suggestions_df):
    results = []
    if not isinstance(suggestions_df, pd.DataFrame) or suggestions_df.empty:
        return "No suggestions to execute."

    selected_trades = suggestions_df[suggestions_df['Select'] == True]
    if selected_trades.empty:
        return "No trades were selected."

    for _, row in selected_trades.iterrows():
        result = place_order(
            tradingsymbol=row['Symbol'], transactiontype=row['Action'],
            quantity=row['Quantity'], ordertype="LIMIT",
            price=row['Price'], exchange=row['Exchange'], producttype="DELIVERY"
        )
        results.append(result)
        time.sleep(SLEEP*5)

    return "\n".join(results)

# **common for automated and manual Buy and Sell**

In [56]:

def place_order(tradingsymbol, transactiontype, quantity, ordertype, price, exchange, producttype):
    result_message = ""
    if not smartApi:
        return "Order not placed: API not connected."

    lookup_map = nse_symbol_token_map if exchange == 'NSE' else bse_symbol_token_map
    token = lookup_map.get(tradingsymbol)
    if not token:
        return f"❗️ Could not find token for {tradingsymbol} on {exchange}."

    try:
        order_params = {
            "variety": "NORMAL", "tradingsymbol": tradingsymbol,
            "symboltoken": str(token), "transactiontype": transactiontype,
            "exchange": exchange, "ordertype": ordertype,
            "producttype": producttype, "duration": "DAY",
            "quantity": str(quantity), "squareoff": "0", "stoploss": "0"
        }
        if ordertype == "LIMIT":
            if price is None or price <= 0:
                return "❗️ Price must be provided for LIMIT orders."
            order_params["price"] = str(round(price, 2))

        order_response = smartApi.placeOrderFullResponse(order_params)

        if isinstance(order_response, dict):
            order_id = None
            if 'data' in order_response and isinstance(order_response['data'], dict):
                order_id = order_response['data'].get('orderid')
            if not order_id:
                order_id = order_response.get('orderid')

            if order_id:
                result_message = f"✅ {transactiontype.upper()} Order placed for {tradingsymbol}! ID: {order_id}"
            else:
                result_message = f"❗️ {transactiontype.upper()} Failed for {tradingsymbol}: {order_response.get('message', 'Unknown API error')}"
        else:
            result_message = f"❗️ {transactiontype.upper()} Failed for {tradingsymbol}: {str(order_response)}"
    except Exception as e:
        result_message = f"❗️ Error placing {transactiontype.upper()} order for {tradingsymbol}: {e}"

    logging.info(result_message)
    return result_message

# **Watchlist**

In [57]:
CSV_PATH = industry
SYMBOL_COLUMN_NAME = "Name"
NEW_COLUMNS = {
    "Remark_AI": "", "Remark_1": "", "Remark_2": "", "Remark_3": "", "Remark_4": "",
    "watchlist_flag": False,
}

# --- Core Data Functions ---

def load_or_initialize_data():
    """
    Loads data and adds necessary columns. Raises an error if the symbol column is not found.
    """
    try:
        df = pd.read_csv(CSV_PATH)
    except FileNotFoundError:
        print(f"Warning: '{CSV_PATH}' not found. Creating a dummy file for demonstration.", file=sys.stderr)
        df = pd.DataFrame({
            SYMBOL_COLUMN_NAME: ['SAMPLE1', 'SAMPLE2'], 'Industry': ['Tech', 'Finance'],
        })

    if SYMBOL_COLUMN_NAME not in df.columns:
        raise ValueError(
            f"Fatal Error: Column '{SYMBOL_COLUMN_NAME}' not found in '{CSV_PATH}'. "
            f"Please check the SYMBOL_COLUMN_NAME variable in the script."
        )

    updated = False
    for col, default_value in NEW_COLUMNS.items():
        if col not in df.columns:
            df[col] = default_value
            updated = True

    if 'watchlist_flag' not in df.columns or df['watchlist_flag'].dtype != bool:
        df['watchlist_flag'] = df.get('watchlist_flag', pd.Series(False)).fillna(False).astype(bool)
        updated = True

    if updated:
        df.to_csv(CSV_PATH, index=False)

    return df

def save_data_to_csv(dataframe):
    """Saves the DataFrame to the CSV file."""
    dataframe.to_csv(CSV_PATH, index=False)
    watchlist_df = dataframe[dataframe["watchlist_flag"] == True]
    return (
        f"✅ Success! Data saved to {CSV_PATH} at {datetime.now().strftime('%H:%M:%S')}",
        watchlist_df
    )

# --- Gradio Interaction Functions ---

def get_initial_views():
    """Loads data and provides initial UI component values."""
    df = load_or_initialize_data()
    symbol_list = df[SYMBOL_COLUMN_NAME].tolist()
    watchlist_df = df[df["watchlist_flag"] == True]
    return df, gr.Dropdown(choices=symbol_list), pd.DataFrame(), watchlist_df

def display_selected_symbol(df_state, selected_symbol):
    """Filters the dataframe to show only the selected symbol's data."""
    if not selected_symbol:
        return pd.DataFrame()
    return df_state[df_state[SYMBOL_COLUMN_NAME] == selected_symbol]

def update_data_on_edit(df_state, selected_symbol, updated_detail_df):
    """
    Handles inline edits by taking the entire updated detail DataFrame as input,
    which bypasses the buggy gr.EditData event object.
    """
    if not selected_symbol or updated_detail_df is None or updated_detail_df.empty:
        return df_state, df_state[df_state["watchlist_flag"] == True]

    updated_row = updated_detail_df.iloc[0]
    original_index = df_state[df_state[SYMBOL_COLUMN_NAME] == selected_symbol].index[0]

    # Update the master dataframe with all values from the edited row
    for col_name, new_value in updated_row.items():
        if col_name in df_state.columns:
            df_state.loc[original_index, col_name] = new_value

    watchlist_df = df_state[df_state["watchlist_flag"] == True]
    return df_state, watchlist_df

def generate_ai_remark(df_state, selected_symbol):
    """Generates a sample AI remark for the selected symbol."""
    if not selected_symbol:
        gr.Warning("Please select a company from the dropdown first.")
        return df_state, pd.DataFrame(), df_state[df_state["watchlist_flag"] == True]

    timestamp = datetime.now().strftime('%Y-%m-%d %H:%M')
    remark = f"AI Note for {selected_symbol}: Monitored on {timestamp}."

    original_index = df_state[df_state[SYMBOL_COLUMN_NAME] == selected_symbol].index[0]
    df_state.loc[original_index, 'Remark_AI'] = remark

    gr.Info(f"AI remark generated for {selected_symbol}.")
    updated_view = df_state[df_state[SYMBOL_COLUMN_NAME] == selected_symbol]
    watchlist_df = df_state[df_state["watchlist_flag"] == True]
    return df_state, updated_view, watchlist_df

def reload_watchlist(_):
    """Reloads the watchlist fresh from CSV (ignores stale memory)."""
    df = load_or_initialize_data()
    watchlist = df[df["watchlist_flag"] == True]
    if watchlist.empty:
        return pd.DataFrame({"Message": ["⭐ No companies in watchlist yet."]})
    return watchlist

def save_watchlist_changes(edited_watchlist):
    """Update master data with edited watchlist and persist to CSV."""
    df = load_or_initialize_data()

    # ensure both are DataFrames
    edited_watchlist = pd.DataFrame(edited_watchlist)

    for _, row in edited_watchlist.iterrows():
        symbol = row[SYMBOL_COLUMN_NAME]

        if symbol in df[SYMBOL_COLUMN_NAME].values:
            for col in df.columns:
                if col in row.index:
                    try:
                        # Cast row[col] to the dtype of df[col]
                        casted_value = row[col]
                        target_dtype = df[col].dtype

                        if pd.isna(casted_value):
                            df.loc[df[SYMBOL_COLUMN_NAME] == symbol, col] = pd.NA
                        else:
                            if target_dtype == "bool":
                                casted_value = str(casted_value).strip().lower() in ["true", "1", "yes"]
                            else:
                                casted_value = target_dtype.type(casted_value)

                            df.loc[df[SYMBOL_COLUMN_NAME] == symbol, col] = casted_value

                    except Exception as e:
                        print(f"⚠️ Skipped casting error for col={col}, value={row[col]}: {e}")

    # persist to CSV
    df.to_csv(CSV_PATH, index=False)

    return "✅ Watchlist changes saved!", df[df["watchlist_flag"] == True]


# **Gradio UI**

In [59]:
load_script_masters()
with gr.Blocks(theme=gr.themes.Glass(), title="SWEET") as iface:
    gr.Markdown("## 📈 SWEET Pathway")
    gr.Markdown("### Smart Wealth Enhancement & Equity Trading Algo")

    with gr.Tabs() as tabs:
        with gr.TabItem("Dashboard"):
            gr.Markdown("## 📊 Dashboard")

            btn = gr.Button("Fetch & Analyze Portfolio", variant="primary")

            summary_table = gr.Dataframe(label="Summary Metrics", interactive=False, wrap=True)

            with gr.Row():
                chart1 = gr.Plot(label=".")
                chart2 = gr.Plot(label=".")

            chart3 = gr.Plot(label=".")

            gr.Markdown("### Holdings Details")
            portfolio_table = gr.Dataframe(label="Portfolio Holdings", interactive=False)

            # New component to handle the CSV download
            download_file = gr.File(label="Download Portfolio CSV", interactive=False)

            # Update the button's click action to populate all outputs
            btn.click(
                fn=analyze_portfolio,
                outputs=[portfolio_table, summary_table, chart1, chart2, chart3, download_file]
            )


        with gr.TabItem("Automated Trading"):
            gr.Markdown("## Rule-Based Trading Suggestions")
            auto_trade_status = gr.Textbox(label="Status", interactive=False)
            analyze_suggest_btn = gr.Button("🔄 Analyze & Generate Suggestions", variant="primary")
            suggestions_df_output = gr.DataFrame(label="Trade Suggestions",
                                                 interactive=True,
                                                 datatype=['bool', 'str', 'str', 'str', 'number', 'number', 'str', 'str'])
            execute_trades_btn = gr.Button("🚀 Execute Selected Trades")
            execution_status = gr.Textbox(label="Execution Results", interactive=False, lines=5)

        with gr.TabItem("Manual Trading"):
            gr.Markdown("## Manual Order Placement")
            manual_trade_status = gr.Textbox(label="Order Status", interactive=False)

            with gr.Row():
                with gr.Group():
                    gr.Markdown("### 🔼 Manual Buy")
                    buy_exchange = gr.Radio(label="Exchange", choices=["NSE", "BSE"], value="NSE")
                    buy_symbol = gr.Dropdown(choices=nse_symbols, label="Symbol")
                    buy_qty = gr.Number(label="Quantity", value=1, minimum=1)
                    buy_product = gr.Radio(label="Product", choices=["DELIVERY"], value="DELIVERY")
                    buy_order_type = gr.Radio(label="Order Type", choices=["MARKET", "LIMIT"], value="MARKET")
                    buy_price = gr.Number(label="Price", value=0, visible=False)
                    place_buy_order_btn = gr.Button("Place Buy Order", variant="primary")

                with gr.Group():
                    gr.Markdown("### 🔽 Manual Sell")
                    sell_exchange = gr.Radio(label="Exchange", choices=["NSE", "BSE"], value="NSE")
                    sell_symbol = gr.Dropdown(choices=nse_symbols, label="Symbol")
                    sell_qty = gr.Number(label="Quantity", value=1, minimum=1)
                    sell_product = gr.Radio(label="Product", choices=["DELIVERY"], value="DELIVERY")
                    sell_order_type = gr.Radio(label="Order Type", choices=["MARKET", "LIMIT"], value="MARKET")
                    sell_price = gr.Number(label="Price", value=0, visible=False)
                    place_sell_order_btn = gr.Button("Place Sell Order", variant="stop")



        with gr.TabItem("Trade Book"):
            gr.Markdown("## Trade Book and Order Book")

            gr.Markdown("## 📑 Your Order Book")
            order_output = gr.DataFrame(label="Order Book", interactive=False)
            get_orders_btn = gr.Button("Get Order Book")
            get_orders_btn.click(get_order_book, outputs=order_output)

            gr.Markdown("### Trade History")
            gr.Markdown("## 📑 Your Trade Book")
            trade_output = gr.DataFrame(label="Trade Book", interactive=False)
            get_trades_btn = gr.Button("Get Trade Book")
            get_trades_btn.click(get_trade_book, outputs=trade_output)


        with gr.TabItem("Master Portfolio"):
            gr.Markdown("## Master Portfolio View, Edit, and Sync")
            status_mp = gr.Textbox(label="Status", interactive=False)

            load_mp_btn = gr.Button("📂 Load Master Portfolio")

            portfolio_df_editor = gr.DataFrame(
                label="Portfolio Editor",
                interactive=True,
                datatype=['str', 'str', 'str', 'number', 'number', 'str', 'number', 'bool']
            )

            with gr.Column():
              save_mp_btn = gr.Button("💾 Save Changes (for manual edits)", variant="primary")
              with gr.Row():
                add_row_btn = gr.Button("➕ Add New Row")
                delete_row_btn = gr.Button("❌ Apply Deletions & Save")

            gr.Markdown("---")
            gr.Markdown("### 🔄 Sync & Verify")
            with gr.Row():
              preview_trades_btn = gr.Button("📥 Preview New Trades from Tradebook")
              confirm_add_btn = gr.Button("Confirm and Add to Portfolio Editor", variant="secondary")

            gr.Markdown("### Preview of New Entries from Trade Book")
            trade_preview_df = gr.DataFrame(label="Trade Preview")

            mismatch_check_btn = gr.Button("🔍 Check Mismatches with Angel One")
            gr.Markdown("### Mismatch Report")
            mismatch_output = gr.Markdown()

            with gr.Accordion("🗄️ Archive", open=True):
              with gr.Row():
                archive_button = gr.Button("🚀 Run Archiving Process", variant="primary")
                view_archive_btn = gr.Button("View Archived Trades")
              archive_df_display = gr.DataFrame(label="Archived Trades")

        with gr.TabItem("Historical Data Downloader"):
            gr.Markdown("### Get Historical Candle Data for Symbols")
            with gr.Row():
                exchange_input = gr.Radio(["NSE", "BSE"], label="Exchange", value="NSE")

            input_method_radio = gr.Radio(["Manual Selection", "Upload Excel File"], label="Select Input Method", value="Manual Selection")

            with gr.Column(visible=True) as manual_input_column:
                symbol_input = gr.Dropdown(choices=nse_symbols, label="Symbols", multiselect=True)
                select_all_btn = gr.Button("Select All Symbols")

            with gr.Column(visible=False) as upload_input_column:
                excel_file_input = gr.File(label="Upload Excel File (with 'Token Symbol' column)", file_types=[".xls", ".xlsx"])

            gr.Markdown("---")

            with gr.Row():
                interval_input = gr.Radio(["ONE_DAY", "FIVE_MINUTE"], value="ONE_DAY")
            with gr.Row():
                from_date_input = gr.Textbox(value=default_from_date, label="From Date")
                to_date_input = gr.Textbox(value=default_to_date, label="To Date")

            with gr.Row():
                get_btn = gr.Button("Get Data")

            output_table = gr.Dataframe()
            download_file = gr.File(label="Download Historical Data")
            status_box_data = gr.Textbox(label="Status", interactive=False)

            exchange_input.change(update_symbol_dropdown, inputs=exchange_input, outputs=symbol_input)
            select_all_btn.click(select_all_symbols, inputs=exchange_input, outputs=symbol_input)

            input_method_radio.change(
                fn=toggle_input,
                inputs=input_method_radio,
                outputs=[manual_input_column, symbol_input, upload_input_column, select_all_btn]
            )

            get_btn.click(get_data_and_prepare_download,
                        inputs=[
                            input_method_radio,
                            symbol_input,
                            excel_file_input,
                            exchange_input,
                            interval_input,
                            from_date_input,
                            to_date_input
                        ],
                        outputs=[output_table, download_file, status_box_data])

            gr.Markdown("### Download the Full Scrip Master List")
            gr.Markdown("This list is useful for finding the correct symbols and token numbers.")
            with gr.Row():
                exchange_master = gr.Radio(["NSE", "BSE"], label="Select Exchange for Master List", value="NSE")
                download_scrip_master_btn = gr.Button("Download Script Master as CSV")
                scrip_master_file = gr.File(label="Download Script Master CSV")
                status_box_master = gr.Textbox(label="Status", interactive=False)

            download_scrip_master_btn.click(
                download_script_master_csv,
                inputs=exchange_master,
                outputs=[scrip_master_file, status_box_master])

        with gr.TabItem("Portfolio"):
            gr.Markdown("## View Your Portfolio")
            portfolio_output = gr.DataFrame(label="Portfolio Holdings", interactive=False)
            download_btn = gr.File(label="Download Portfolio CSV")

            get_portfolio_btn = gr.Button("Get Portfolio Holdings")
            get_portfolio_btn.click(
                get_portfolio_holdings,
                outputs=[portfolio_output, download_btn]
            )






        with gr.TabItem("Watchlist Manager"):
          gr.Markdown("# 📈 Watchlist Manager")

          df_master_state = gr.State()

          with gr.Row():
            symbol_dropdown = gr.Dropdown(label="Select a Company to View/Edit", scale=3)
            generate_btn = gr.Button("🤖 Generate AI Remark", variant="secondary", scale=1)
            save_btn = gr.Button("💾 Save All Changes", variant="primary", scale=1)

          status_label = gr.Label(show_label=False)

          gr.Markdown("---")
          gr.Markdown("### ✏️ Edit Selected Company Details")
          symbol_detail_view = gr.DataFrame(interactive=True, wrap=False)

          gr.Markdown("---")
          gr.Markdown("### ⭐ Your Watchlist")
          with gr.Row():
            save_watchlist_btn = gr.Button("💾 Save Watchlist Changes", variant="primary")
            load_watchlist_btn = gr.Button("🔄 Load Watchlist", variant="secondary")
          watchlist_df_view = gr.DataFrame(interactive=True,wrap=False)

          iface.load(
              fn=get_initial_views,
              outputs=[df_master_state, symbol_dropdown, symbol_detail_view, watchlist_df_view]
          )
          symbol_dropdown.change(
              fn=display_selected_symbol,
              inputs=[df_master_state, symbol_dropdown],
              outputs=[symbol_detail_view]
          )

          symbol_detail_view.input(
              fn=update_data_on_edit,
              inputs=[df_master_state, symbol_dropdown, symbol_detail_view],
              outputs=[df_master_state, watchlist_df_view]
          )

          generate_btn.click(
              fn=generate_ai_remark,
              inputs=[df_master_state, symbol_dropdown],
              outputs=[df_master_state, symbol_detail_view, watchlist_df_view]
          )
          save_btn.click(
              fn=save_data_to_csv,
              inputs=[df_master_state],
              outputs=[status_label, watchlist_df_view]
          )

          load_watchlist_btn.click(
              fn=reload_watchlist,
              inputs=[df_master_state],
              outputs=[watchlist_df_view]
          )

          save_watchlist_btn.click(
              fn=save_watchlist_changes,
              inputs=[watchlist_df_view],
              outputs=[status_label, watchlist_df_view]
          )
    gr.Markdown("## SWEET Pathway : Where smart trading meets SWEET success.")
    gr.Markdown("Conceptualized & Developed by Rohit Jain & Tanmay Jain")


    # --- 7. Event Handlers ---

    # Generic UI helpers
    def toggle_price_visibility(order_type):
        return gr.update(visible=order_type == "LIMIT")

    def update_symbol_dropdown(exchange):
        symbol_list = nse_symbols_list if exchange == "NSE" else bse_symbols_list
        return gr.update(choices=symbol_list)

    buy_order_type.change(fn=toggle_price_visibility, inputs=buy_order_type, outputs=buy_price)
    sell_order_type.change(fn=toggle_price_visibility, inputs=sell_order_type, outputs=sell_price)
    buy_exchange.change(fn=update_symbol_dropdown, inputs=buy_exchange, outputs=buy_symbol)
    sell_exchange.change(fn=update_symbol_dropdown, inputs=sell_exchange, outputs=sell_symbol)

    # Manual Trading
    place_buy_order_btn.click(fn=place_order, inputs=[buy_symbol, gr.Textbox("BUY", visible=False), buy_qty, buy_order_type, buy_price, buy_exchange, buy_product], outputs=[manual_trade_status])
    place_sell_order_btn.click(fn=place_order, inputs=[sell_symbol, gr.Textbox("SELL", visible=False), sell_qty, sell_order_type, sell_price, sell_exchange, sell_product], outputs=[manual_trade_status])

    # Use the new function to load the portfolio with checkboxes
    load_mp_btn.click(fn=load_and_display_portfolio, outputs=[portfolio_df_editor, status_mp])

    # Save button is for manual cell edits
    save_mp_btn.click(fn=save_portfolio_to_csv, inputs=[portfolio_df_editor], outputs=[status_mp])

    # Add row function
    def add_row_to_df(df):
        # Create a new row that matches the existing columns, including 'Delete'
        new_row_data = {col: '' for col in df.columns}
        if 'Delete' in new_row_data:
            new_row_data['Delete'] = False # Default new rows to not be deleted
        new_row_df = pd.DataFrame([new_row_data])
        return pd.concat([df, new_row_df], ignore_index=True)
    add_row_btn.click(fn=add_row_to_df, inputs=portfolio_df_editor, outputs=portfolio_df_editor)

    # Connect the new delete button to the new delete_and_save_rows function
    delete_row_btn.click(
        fn=delete_and_save_rows,
        inputs=[portfolio_df_editor],
        outputs=[portfolio_df_editor, status_mp]
    )

    # Other Dashboard Handlers
    mismatch_check_btn.click(fn=check_mismatches, outputs=[mismatch_output])
    view_archive_btn.click(lambda: load_archive_from_csv(ARCHIVE_FILE), outputs=[archive_df_display])
    preview_trades_btn.click(fn=preview_trades_from_tradebook, outputs=[trade_preview_df, status_mp])
    confirm_add_btn.click(fn=add_preview_to_portfolio, inputs=[portfolio_df_editor, trade_preview_df], outputs=[portfolio_df_editor, status_mp])

    # Automated Trading Handlers
    analyze_suggest_btn.click(fn=on_analyze_and_suggest_click, outputs=[auto_trade_status, suggestions_df_output, portfolio_df_editor])
    execute_trades_btn.click(fn=execute_selected_trades, inputs=[suggestions_df_output], outputs=[execution_status])

    archive_button.click(fn=run_archiving_process,inputs=None)



In [60]:
if __name__ == "__main__":
    iface.launch(debug = False, inline= False)

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://9c4ea0fb13b5319ef0.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
